In [119]:
import sklearn
import numpy as np
import os
import pandas as pd
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import missingno as msno
import optuna
import gc
from prettytable import PrettyTable
import seaborn as sns
import lightgbm as lgb
import plotly.express as px
from joblib import Parallel, delayed
from tqdm import tqdm
from numexpr import evaluate
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.model_selection import KFold
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import PowerTransformer, FunctionTransformer, MinMaxScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from scipy.stats import uniform, randint
import tensorflow
import keras
import pandas as pd
import numpy as np
from scipy.stats import pointbiserialr, spearmanr
from skopt import BayesSearchCV
from sklearn.impute import KNNImputer
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, CatBoostClassifier
import warnings
from scipy.stats import pearsonr, chi2_contingency, pointbiserialr
warnings.filterwarnings("ignore")

In [120]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [121]:
print(train.shape, test.shape)

(165034, 14) (110023, 13)


In [122]:
train.columns

Index(['id', 'CustomerId', 'Surname', 'CreditScore', 'Geography', 'Gender',
       'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [123]:
train.drop(columns=["id", "Surname", "CustomerId"],inplace=True)
test.drop(columns=["id", "Surname"],inplace=True)

In [124]:
train.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

In [125]:
train_copy = train.copy()
test_copy  = test.copy()

In [126]:
target='Exited'

In [127]:
y_target=train_copy['Exited']
y_target

0         0
1         0
2         0
3         0
4         0
         ..
165029    0
165030    0
165031    0
165032    0
165033    1
Name: Exited, Length: 165034, dtype: int64

In [128]:
train_copy

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0
...,...,...,...,...,...,...,...,...,...,...,...
165029,667,Spain,Female,33.0,2,0.00,1,1.0,1.0,131834.75,0
165030,792,France,Male,35.0,3,0.00,1,0.0,0.0,131834.45,0
165031,565,France,Male,31.0,5,0.00,1,1.0,1.0,127429.56,0
165032,554,Spain,Female,30.0,7,161533.00,1,0.0,1.0,71173.03,0


In [129]:
train_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165034 entries, 0 to 165033
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   CreditScore      165034 non-null  int64  
 1   Geography        165034 non-null  object 
 2   Gender           165034 non-null  object 
 3   Age              165034 non-null  float64
 4   Tenure           165034 non-null  int64  
 5   Balance          165034 non-null  float64
 6   NumOfProducts    165034 non-null  int64  
 7   HasCrCard        165034 non-null  float64
 8   IsActiveMember   165034 non-null  float64
 9   EstimatedSalary  165034 non-null  float64
 10  Exited           165034 non-null  int64  
dtypes: float64(5), int64(4), object(2)
memory usage: 13.9+ MB


In [130]:
train_copy.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000
mean,656.454373,38.125888,5.020353,55478.086689,1.554455,0.753954,0.497770,112574.822734,0.211599
std,80.103340,8.867205,2.806159,62817.663278,0.547154,0.430707,0.499997,50292.865585,0.408443
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,597.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,74637.570000,0.000000
50%,659.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,117948.000000,0.000000
75%,710.000000,42.000000,7.000000,119939.517500,2.000000,1.000000,1.000000,155152.467500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000


In [131]:
train_copy.nunique()

CreditScore          457
Geography              3
Gender                 2
Age                   71
Tenure                11
Balance            30075
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary    55298
Exited                 2
dtype: int64

In [132]:
#cols_to_object = ["Tenure", "NumOfProducts"]
#train_copy[cols_to_object] = train_copy[cols_to_object].astype("object")
#train_copy.info()

In [133]:
train_copy.describe(include = "object")

,Geography,Gender
count,165034,165034
unique,3,2
top,France,Male
freq,94215,93150


In [134]:
train_copy.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000,165034.000000
mean,656.454373,38.125888,5.020353,55478.086689,1.554455,0.753954,0.497770,112574.822734,0.211599
std,80.103340,8.867205,2.806159,62817.663278,0.547154,0.430707,0.499997,50292.865585,0.408443
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,597.000000,32.000000,3.000000,0.000000,1.000000,1.000000,0.000000,74637.570000,0.000000
50%,659.000000,37.000000,5.000000,0.000000,2.000000,1.000000,0.000000,117948.000000,0.000000
75%,710.000000,42.000000,7.000000,119939.517500,2.000000,1.000000,1.000000,155152.467500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.000000,1.000000,199992.480000,1.000000


In [135]:
Xtrain_copy = train_copy.drop(target, axis =1)
Xtrain_copy

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97
1,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50
2,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69
3,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88
4,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83
...,...,...,...,...,...,...,...,...,...,...
165029,667,Spain,Female,33.0,2,0.00,1,1.0,1.0,131834.75
165030,792,France,Male,35.0,3,0.00,1,0.0,0.0,131834.45
165031,565,France,Male,31.0,5,0.00,1,1.0,1.0,127429.56
165032,554,Spain,Female,30.0,7,161533.00,1,0.0,1.0,71173.03


In [136]:
table = PrettyTable()

In [137]:
table.field_names = ['Feature', 'Data Type', 'Train Missing %', 'Test Missing %']
table

Feature,Data Type,Train Missing %,Test Missing %


In [138]:
null_percent_train=(train_copy.isnull().mean()*100).round(1)
null_percent_train

CreditScore        0.0
Geography          0.0
Gender             0.0
Age                0.0
Tenure             0.0
Balance            0.0
NumOfProducts      0.0
HasCrCard          0.0
IsActiveMember     0.0
EstimatedSalary    0.0
Exited             0.0
dtype: float64

In [139]:
null_percent_test=((test_copy.isnull().mean())*100).round(1)
null_percent_test

CustomerId         0.0
CreditScore        0.0
Geography          0.0
Gender             0.0
Age                0.0
Tenure             0.0
Balance            0.0
NumOfProducts      0.0
HasCrCard          0.0
IsActiveMember     0.0
EstimatedSalary    0.0
dtype: float64

In [140]:
# Création du tableau avec PrettyTable
table = PrettyTable(['Column', 'Data Type', 'Non-null % Train', 'Non-null % Test'])
table 

Column,Data Type,Non-null % Train,Non-null % Test


In [141]:
for column in train_copy.columns:
    data_type = str(train_copy[column].dtype)
    non_null_train = null_percent_train[column]
    non_null_test = null_percent_test[column] if column != target else '-'
    table.add_row([column, data_type, non_null_train, non_null_test])

In [142]:
table

Column,Data Type,Non-null % Train,Non-null % Test
CreditScore,int64,0.0,0.0
Geography,object,0.0,0.0
Gender,object,0.0,0.0
Age,float64,0.0,0.0
Tenure,int64,0.0,0.0
Balance,float64,0.0,0.0
NumOfProducts,int64,0.0,0.0
HasCrCard,float64,0.0,0.0
IsActiveMember,float64,0.0,0.0
EstimatedSalary,float64,0.0,0.0


In [171]:
from sklearn.compose import make_column_selector as selector

In [172]:
cat_cols_to_encode = ["Geography", "Tenure", "NumOfProducts", "Gender"]
cat_cols_to_encode

['Geography', 'Tenure', 'NumOfProducts', 'Gender']

In [173]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(Xtrain_copy, y_target,
                                                        test_size=0.2,
                                                        random_state=0,
                                                        stratify=y_target)

In [174]:
print(Xtrain.shape, Xtest.shape)

(132027, 10) (33007, 10)


In [146]:
def min_max_scaler(train, test, colonne):
    '''
    Met à l'échelle la colonne spécifiée dans les ensembles de données d'entraînement et de test en utilisant la mise à l'échelle Min-Max.
    '''
    # Initialiser MinMaxScaler
    sc = MinMaxScaler()
    # Regrouper les données d'entraînement et de test pour déterminer les valeurs min et max globales
    data = pd.concat([train[[colonne]], test[[colonne]]])  # Combiner les données d'entraînement et de test
    sc.fit(data)  # Ajuster le scaler sur les données combinées
    # Transformer la colonne dans les ensembles de données d'entraînement et de test
    train[colonne] = sc.transform(train[[colonne]])
    test[colonne] = sc.transform(test[[colonne]])
    return train, test

In [147]:
cont_cols = list(Xtrain.select_dtypes(include = ["int", "float"]).columns)
cont_cols

['CreditScore',
 'Age',
 'Tenure',
 'Balance',
 'NumOfProducts',
 'HasCrCard',
 'IsActiveMember',
 'EstimatedSalary']

In [148]:
Xtrain.nunique()

CreditScore          455
Geography              3
Gender                 2
Age                   71
Tenure                11
Balance            25646
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary    47156
dtype: int64

In [149]:
Xtrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132027 entries, 116180 to 7704
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   CreditScore      132027 non-null  int64  
 1   Geography        132027 non-null  object 
 2   Gender           132027 non-null  object 
 3   Age              132027 non-null  float64
 4   Tenure           132027 non-null  int64  
 5   Balance          132027 non-null  float64
 6   NumOfProducts    132027 non-null  int64  
 7   HasCrCard        132027 non-null  float64
 8   IsActiveMember   132027 non-null  float64
 9   EstimatedSalary  132027 non-null  float64
dtypes: float64(5), int64(3), object(2)
memory usage: 11.1+ MB


In [150]:
### Traitement des données manquantes 

In [151]:
def store_missing_rows(df, features):
    '''Fonction qui stocke les lignes contenant des valeurs manquantes pour un ensemble donné de fonctionnalités'''
    missing_rows = {}
    # Grouper les lignes par feature manquante et collecter les index des lignes
    for feature in features:
        missing_indices = df.index[df[feature].isnull()].tolist()
        if missing_indices:
            missing_rows[feature] = missing_indices
    return missing_rows

In [38]:
m = store_missing_rows(Xtrain, cat_cols_to_encode)
m

{}

In [39]:
train.columns

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'Exited'],
      dtype='object')

In [40]:
""" 
def fill_missing_numerical(train, test, target, max_iterations=10):
    '''Iterative Missing Imputer: Updates filled missing values iteratively using various regression models'''
    # Copie du DataFrame train
    train_temp = train.copy()
    if target in train_temp.columns:
        train_temp = train_temp.drop(columns=target)
    # Concaténation de train et test
    df = pd.concat([train_temp, test], axis="rows").reset_index(drop=True)
    # Sélection des caractéristiques avec des valeurs manquantes
    features = df.columns[df.isnull().any()]
    # Vérification de la présence de caractéristiques avec des valeurs manquantes
    if len(features) > 0:
        # Boucle sur le nombre d'itérations
        for iteration in range(max_iterations):
            for feature in features:
                # Sélection des lignes avec des valeurs manquantes pour la caractéristique
                rows_miss = df[df[feature].isnull()].index

                # Sélection des données avec et sans valeurs manquantes
                missing_temp = df.loc[rows_miss].copy()
                non_missing_temp = df.drop(index=rows_miss).copy()
                
                # Séparation des caractéristiques indépendantes et dépendantes
                X_train = non_missing_temp.drop(columns=[feature])
                y_train = non_missing_temp[feature]
                
                # Choix du modèle de régression
                models = {
                    'LightGBM': lgb.LGBMRegressor(),
                    'RandomForest': RandomForestRegressor(),
                    'LinearRegression': LinearRegression()
                }
                
                best_model = None
                best_score = -np.inf
                
                # Entrainement et validation des modèles
                for name, model in models.items():
                    scores = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
                    mean_score = np.mean(scores)
                    
                    if mean_score > best_score:
                        best_score = mean_score
                        best_model = model
                
                # Entrainement final du meilleur modèle
                best_model.fit(X_train, y_train)

                # Prédiction des valeurs manquantes
                y_pred = best_model.predict(missing_temp.drop(columns=[feature]))
                
                # Mise à jour des valeurs manquantes dans le DataFrame
                df.loc[rows_miss, feature] = y_pred

    # Séparation des données train et test
    train[features] = df.iloc[:train.shape[0]][features].values
    test[features] = df.iloc[train.shape[0]:][features].values

    return train, test
"""


' \ndef fill_missing_numerical(train, test, target, max_iterations=10):\n    \'\'\'Iterative Missing Imputer: Updates filled missing values iteratively using various regression models\'\'\'\n    # Copie du DataFrame train\n    train_temp = train.copy()\n    if target in train_temp.columns:\n        train_temp = train_temp.drop(columns=target)\n    # Concaténation de train et test\n    df = pd.concat([train_temp, test], axis="rows").reset_index(drop=True)\n    # Sélection des caractéristiques avec des valeurs manquantes\n    features = df.columns[df.isnull().any()]\n    # Vérification de la présence de caractéristiques avec des valeurs manquantes\n    if len(features) > 0:\n        # Boucle sur le nombre d\'itérations\n        for iteration in range(max_iterations):\n            for feature in features:\n                # Sélection des lignes avec des valeurs manquantes pour la caractéristique\n                rows_miss = df[df[feature].isnull()].index\n\n                # Sélection des

In [41]:
def fill_missing(train, test, target, max_iterations=10):
    '''
    Iterative Missing Imputer: Updates filled missing values iteratively using KNN 
    imputation for numerical variables and mode imputation for categorical variables
    
    '''
    # Copie du DataFrame train
    train_temp = train.copy()
    if target in train_temp.columns:
        train_temp = train_temp.drop(columns=target)
    
    # Concaténation de train et test
    df = pd.concat([train_temp, test], axis="rows").reset_index(drop=True)
    
    # Sélection des caractéristiques avec des valeurs manquantes
    numeric_features = df.select_dtypes(include=np.number).columns
    categorical_features = df.select_dtypes(include='object').columns
    
    # Imputation des valeurs manquantes pour les variables numériques avec KNNImputer
    if len(numeric_features) > 0:
        imputer_numeric = KNNImputer()
        df[numeric_features] = imputer_numeric.fit_transform(df[numeric_features])
    
    # Imputation des valeurs manquantes pour les variables catégorielles avec le mode
    if len(categorical_features) > 0:
        for feature in categorical_features:
            mode_value = df[feature].mode()[0]
            df[feature] = df[feature].fillna(mode_value)
    
    # Séparation des données train et test
    train[numeric_features] = df.iloc[:train.shape[0]][numeric_features].values
    test[numeric_features] = df.iloc[train.shape[0]:][numeric_features].values
    
    train[categorical_features] = df.iloc[:train.shape[0]][categorical_features].values
    test[categorical_features] = df.iloc[train.shape[0]:][categorical_features].values
    return train, test

In [42]:
#Xtrain_miss, Xtest_miss = Xtrain.copy(), Xtest.copy()

In [43]:
Xtrain, Xtest = fill_missing(Xtrain, Xtest, target, max_iterations=10)

In [44]:
Xtrain

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
116180,750.0,France,Female,47.0,6.0,121376.15,2.0,1.0,1.0,54473.60
152256,663.0,France,Male,40.0,9.0,0.00,2.0,0.0,1.0,25165.36
124302,709.0,Spain,Female,42.0,5.0,0.00,2.0,1.0,1.0,151344.23
79682,683.0,France,Male,32.0,6.0,0.00,2.0,1.0,1.0,79919.13
127888,776.0,Spain,Male,36.0,6.0,0.00,2.0,0.0,0.0,63908.86
...,...,...,...,...,...,...,...,...,...,...
144921,575.0,France,Male,39.0,2.0,0.00,2.0,1.0,0.0,81259.25
41674,634.0,Spain,Female,32.0,3.0,0.00,2.0,1.0,0.0,88659.67
73618,842.0,France,Female,38.0,1.0,141786.57,1.0,1.0,0.0,104077.01
122092,591.0,France,Female,30.0,9.0,92711.66,1.0,0.0,1.0,172363.46


In [45]:
Xtest

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
59556,515.0,France,Male,34.0,5.0,0.00,2.0,1.0,0.0,27022.57
18893,743.0,France,Female,41.0,4.0,0.00,2.0,1.0,1.0,190911.02
38472,628.0,France,Male,28.0,4.0,0.00,2.0,1.0,0.0,49594.02
53391,461.0,Spain,Female,35.0,5.0,0.00,1.0,1.0,0.0,133814.28
108842,494.0,France,Male,35.0,9.0,115569.21,2.0,1.0,1.0,15378.75
...,...,...,...,...,...,...,...,...,...,...
30644,586.0,France,Female,28.0,7.0,103522.75,1.0,0.0,0.0,96833.00
115675,516.0,France,Male,32.0,6.0,0.00,2.0,1.0,0.0,88890.05
4440,486.0,France,Female,35.0,8.0,97819.36,1.0,0.0,1.0,120586.32
43916,743.0,Spain,Male,36.0,1.0,0.00,1.0,1.0,0.0,195192.40


In [46]:
Xtest.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
dtype: int64

In [47]:
# Fonction pour obtenir la valeur la plus proche dans une liste
def nearest_val(target, common):
    return min(common, key=lambda x: abs(x - target))

# Fonction pour traiter les valeurs catégorielles rares

def handle_rare_categories(train, test, col_list):
    for col in col_list:
        uncommon = set(test[col].unique()) | set(train[col].unique())
        common = set(test[col].unique()) & set(train[col].unique())
        uncommon -= common
        if uncommon:
            train[col] = train[col].apply(lambda x: nearest_val(x, common) if x not in common else x)
            test[col] = test[col].apply(lambda x: nearest_val(x, common) if x not in common else x)
    return train, test

In [48]:
Xtrain.nunique()

CreditScore          455
Geography              3
Gender                 2
Age                   71
Tenure                11
Balance            25646
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary    47156
dtype: int64

In [49]:
cols_rare = ["Tenure", "NumOfProducts"]

In [50]:
Xtrain, Xtest = handle_rare_categories(Xtrain, Xtest, cols_rare)

In [51]:
list(Xtrain) == list(Xtest)

True

In [52]:
Xtrain.nunique()

CreditScore          455
Geography              3
Gender                 2
Age                   71
Tenure                11
Balance            25646
NumOfProducts          4
HasCrCard              2
IsActiveMember         2
EstimatedSalary    47156
dtype: int64

In [152]:
def OHE(train, test, col, min_percentage=0.0):
    """
    One-Hot Encoding function optimized for feature encoding in train and test datasets.
    Removes the category with the least frequency within each column if it represents less than min_percentage.

    Parameters:
    train (pd.DataFrame): Training dataset.
    test (pd.DataFrame): Test dataset.
    cols (list): List of column names to perform one-hot encoding.
    target_col (str): Name of the target column (to be excluded from test after encoding).
    min_percentage (float): Minimum percentage threshold for category frequency. Default is 0.0.

    Returns:
    pd.DataFrame: Encoded training dataset.
    pd.DataFrame: Encoded test dataset.
    """

    # Copy original dataframes to avoid modifying input data
    train_encode = pd.DataFrame(index=train.index)
    test_encode = pd.DataFrame(index=test.index)
    ohe = "OHE"

    train_value_counts = train[col].value_counts(normalize=True)
    min_freq_category = train_value_counts.idxmin()
    min_freq_percentage = train_value_counts.min()

    if min_freq_percentage < min_percentage:
        min_freq_category_name = f"{col}_{ohe}_{min_freq_category}"
        train_encode = pd.concat([
            train_encode,
            pd.get_dummies(train[col], prefix=f"{col}_OHE", prefix_sep='_')
            .drop(columns=min_freq_category_name)
        ], axis=1)

        test_encode = pd.concat([
            test_encode,
            pd.get_dummies(test[col], prefix=f"{col}_OHE", prefix_sep='_')
            .drop(columns=min_freq_category_name)
        ], axis=1)
    else:
        train_encode = pd.concat([
            train_encode,
            pd.get_dummies(train[col], prefix=f"{col}_OHE", prefix_sep='_')
        ], axis=1)

        test_encode = pd.concat([
            test_encode,
            pd.get_dummies(test[col], prefix=f"{col}_OHE", prefix_sep='_')
        ], axis=1)
    return train_encode, test_encode

In [153]:
Xtrain

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
116180,750,France,Female,47.0,6,121376.15,2,1.0,1.0,54473.60
152256,663,France,Male,40.0,9,0.00,2,0.0,1.0,25165.36
124302,709,Spain,Female,42.0,5,0.00,2,1.0,1.0,151344.23
79682,683,France,Male,32.0,6,0.00,2,1.0,1.0,79919.13
127888,776,Spain,Male,36.0,6,0.00,2,0.0,0.0,63908.86
...,...,...,...,...,...,...,...,...,...,...
144921,575,France,Male,39.0,2,0.00,2,1.0,0.0,81259.25
41674,634,Spain,Female,32.0,3,0.00,2,1.0,0.0,88659.67
73618,842,France,Female,38.0,1,141786.57,1,1.0,0.0,104077.01
122092,591,France,Female,30.0,9,92711.66,1,0.0,1.0,172363.46


In [154]:
cat_cols_to_encode

['Geography', 'Tenure', 'NumOfProducts', 'Gender']

In [155]:
Xtrain

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
116180,750,France,Female,47.0,6,121376.15,2,1.0,1.0,54473.60
152256,663,France,Male,40.0,9,0.00,2,0.0,1.0,25165.36
124302,709,Spain,Female,42.0,5,0.00,2,1.0,1.0,151344.23
79682,683,France,Male,32.0,6,0.00,2,1.0,1.0,79919.13
127888,776,Spain,Male,36.0,6,0.00,2,0.0,0.0,63908.86
...,...,...,...,...,...,...,...,...,...,...
144921,575,France,Male,39.0,2,0.00,2,1.0,0.0,81259.25
41674,634,Spain,Female,32.0,3,0.00,2,1.0,0.0,88659.67
73618,842,France,Female,38.0,1,141786.57,1,1.0,0.0,104077.01
122092,591,France,Female,30.0,9,92711.66,1,0.0,1.0,172363.46


In [156]:
OHE_Xtrain, OHE_Xtest = OHE(Xtrain, Xtest, "Tenure", min_percentage=0.005)
OHE_Xtrain

,Tenure_OHE_0,Tenure_OHE_1,Tenure_OHE_2,Tenure_OHE_3,Tenure_OHE_4,Tenure_OHE_5,Tenure_OHE_6,Tenure_OHE_7,Tenure_OHE_8,Tenure_OHE_9,Tenure_OHE_10
116180,0,0,0,0,0,0,1,0,0,0,0
152256,0,0,0,0,0,0,0,0,0,1,0
124302,0,0,0,0,0,1,0,0,0,0,0
79682,0,0,0,0,0,0,1,0,0,0,0
127888,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
144921,0,0,1,0,0,0,0,0,0,0,0
41674,0,0,0,1,0,0,0,0,0,0,0
73618,0,1,0,0,0,0,0,0,0,0,0
122092,0,0,0,0,0,0,0,0,0,1,0


In [157]:
 OHE_Xtest

,Tenure_OHE_0,Tenure_OHE_1,Tenure_OHE_2,Tenure_OHE_3,Tenure_OHE_4,Tenure_OHE_5,Tenure_OHE_6,Tenure_OHE_7,Tenure_OHE_8,Tenure_OHE_9,Tenure_OHE_10
59556,0,0,0,0,0,1,0,0,0,0,0
18893,0,0,0,0,1,0,0,0,0,0,0
38472,0,0,0,0,1,0,0,0,0,0,0
53391,0,0,0,0,0,1,0,0,0,0,0
108842,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
30644,0,0,0,0,0,0,0,1,0,0,0
115675,0,0,0,0,0,0,1,0,0,0,0
4440,0,0,0,0,0,0,0,0,1,0,0
43916,0,1,0,0,0,0,0,0,0,0,0


In [158]:
list(OHE_Xtrain.columns) + ["Tenure"]

['Tenure_OHE_0',
 'Tenure_OHE_1',
 'Tenure_OHE_2',
 'Tenure_OHE_3',
 'Tenure_OHE_4',
 'Tenure_OHE_5',
 'Tenure_OHE_6',
 'Tenure_OHE_7',
 'Tenure_OHE_8',
 'Tenure_OHE_9',
 'Tenure_OHE_10',
 'Tenure']

In [160]:
def cat_encoding(Xtrain, Xtest, cols_to_encode):
    Xtrain_encode = Xtrain.copy()
    Xtest_encode = Xtest.copy()
    for col in cols_to_encode:
        OHE_train, OHE_test = OHE(Xtrain, Xtest, col, min_percentage=0.005)
        Xtrain_encode = pd.concat([Xtrain_encode, OHE_train], axis = 1).drop(col, axis = 1)
        Xtest_encode = pd.concat([Xtest_encode, OHE_test], axis = 1).drop(col, axis = 1)    
    return   Xtrain_encode, Xtest_encode

In [161]:
Xtrain_enc, Xtest_enc = cat_encoding(Xtrain, Xtest, cat_cols_to_encode)
Xtrain_enc

,CreditScore,Age,Balance,HasCrCard,IsActiveMember,EstimatedSalary,Geography_OHE_France,Geography_OHE_Germany,Geography_OHE_Spain,Tenure_OHE_0,...,Tenure_OHE_6,Tenure_OHE_7,Tenure_OHE_8,Tenure_OHE_9,Tenure_OHE_10,NumOfProducts_OHE_1,NumOfProducts_OHE_2,NumOfProducts_OHE_3,Gender_OHE_Female,Gender_OHE_Male
116180,750,47.0,121376.15,1.0,1.0,54473.60,1,0,0,0,...,1,0,0,0,0,0,1,0,1,0
152256,663,40.0,0.00,0.0,1.0,25165.36,1,0,0,0,...,0,0,0,1,0,0,1,0,0,1
124302,709,42.0,0.00,1.0,1.0,151344.23,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
79682,683,32.0,0.00,1.0,1.0,79919.13,1,0,0,0,...,1,0,0,0,0,0,1,0,0,1
127888,776,36.0,0.00,0.0,0.0,63908.86,0,0,1,0,...,1,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144921,575,39.0,0.00,1.0,0.0,81259.25,1,0,0,0,...,0,0,0,0,0,0,1,0,0,1
41674,634,32.0,0.00,1.0,0.0,88659.67,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
73618,842,38.0,141786.57,1.0,0.0,104077.01,1,0,0,0,...,0,0,0,0,0,1,0,0,1,0
122092,591,30.0,92711.66,0.0,1.0,172363.46,1,0,0,0,...,0,0,0,1,0,1,0,0,1,0


In [162]:
Xtrain_enc.columns

Index(['CreditScore', 'Age', 'Balance', 'HasCrCard', 'IsActiveMember',
       'EstimatedSalary', 'Geography_OHE_France', 'Geography_OHE_Germany',
       'Geography_OHE_Spain', 'Tenure_OHE_0', 'Tenure_OHE_1', 'Tenure_OHE_2',
       'Tenure_OHE_3', 'Tenure_OHE_4', 'Tenure_OHE_5', 'Tenure_OHE_6',
       'Tenure_OHE_7', 'Tenure_OHE_8', 'Tenure_OHE_9', 'Tenure_OHE_10',
       'NumOfProducts_OHE_1', 'NumOfProducts_OHE_2', 'NumOfProducts_OHE_3',
       'Gender_OHE_Female', 'Gender_OHE_Male'],
      dtype='object')

In [163]:
list(Xtrain_enc) == list(Xtest_enc)

True

In [164]:
cont_cols = ['CreditScore',
             'Age',
             'Balance',
             'EstimatedSalary']
cont_cols 

['CreditScore', 'Age', 'Balance', 'EstimatedSalary']

In [165]:
def apply_transformations(train, test, col, transformations):
    for transformation in transformations:
        name = transformation["name"]
        func = transformation["func"]
        train[name] = func(train[col].to_numpy().reshape(-1, 1)) 
        test[name] = func(test[col].to_numpy().reshape(-1, 1)) 
        
def transformer(Xtrain, Xtest, Ytrain, cont_cols, target):
    global unimportant_features, overall_best_score, overall_best_col
    unimportant_features = []
    overall_best_score = 0
    overall_best_col = 'none'
    table = pd.DataFrame(columns=['Feature', 'Initial ROC_AUC', 'Transformation', 'Transformed ROC_AUC'])

    for col in cont_cols:
        # {"name": "bx_cx_" + col, "func": lambda x: PowerTransformer(method='box-cox').fit_transform(x)},
        transformations = [
            {"name": "log_" + col, "func": lambda x: np.log1p(x)},
            {"name": "sqrt_" + col, "func": lambda x: np.sqrt(x)},
            {"name": "y_J_" + col, "func": lambda x: PowerTransformer(method='yeo-johnson').fit_transform(x)},
            {"name": "pow_" + col, "func": lambda x: FunctionTransformer(lambda x: np.power(x + 1 - np.min(x), 0.25)).fit_transform(x)},
            {"name": "pow2_" + col, "func": lambda x: FunctionTransformer(lambda x: np.power(x + 1 - np.min(x), 2)).fit_transform(x)},
            {"name": "log_sqrt" + col, "func": lambda x: np.log1p(np.sqrt(x))}
        ]
        #Xtrain, Xtest = fill_missing_numerical(Xtrain, Xtest, target, max_iterations=10)
        apply_transformations(Xtrain, Xtest, col, transformations) 
        
        temp_cols = [transformation["name"] for transformation in transformations] + [col]
        auc_scores = []
        
        for f in temp_cols:
            X = Xtrain[[f]]
            y = Ytrain            
            kf = KFold(n_splits=5, shuffle=True, random_state=42)
            auc = []
            for train_idx, val_idx in kf.split(X, y):
                X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
                X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]                
                model = LogisticRegression()
                model.fit(X_train, y_train)
                y_pred = model.predict_proba(X_val)[:,1]
                auc.append(roc_auc_score(y_val, y_pred))
            auc_scores.append((f, np.mean(auc)))
        best_col, best_auc = sorted(auc_scores, key=lambda x: x[1], reverse=True)[0]
        
        if best_auc > overall_best_score:
            overall_best_score = best_auc
            overall_best_col = best_col
            
        cols_to_drop = [f for f in temp_cols if f != best_col]
        final_selection = [f for f in temp_cols if f not in cols_to_drop]
        
        if cols_to_drop:
            unimportant_features += cols_to_drop
        table.loc[len(table)] = [col, np.mean(auc), best_col, best_auc]
        
    Xtrain = Xtrain.drop(unimportant_features, axis = 1)
    Xtest  = Xtest.drop(unimportant_features, axis = 1)

    print(table)
    print("Overall best CV ROC AUC score:", overall_best_score)
    return Xtrain, Xtest, unimportant_features

In [166]:
Xtrain_trans, Xtest_trans, unimportant_features = transformer(Xtrain_enc, Xtest_enc, Ytrain, cont_cols, target)

           Feature  Initial ROC_AUC       Transformation  Transformed ROC_AUC
0      CreditScore         0.523246      log_CreditScore             0.523246
1              Age         0.749407              log_Age             0.749407
2          Balance         0.581664          log_Balance             0.581664
3  EstimatedSalary         0.485075  log_EstimatedSalary             0.514925
Overall best CV ROC AUC score: 0.7494065112708477


In [68]:
print(Xtrain_trans.shape, Xtest_trans.shape)

(132027, 25) (33007, 25)


In [69]:
Xtrain_trans.columns

Index(['HasCrCard', 'IsActiveMember', 'Geography_OHE_France',
       'Geography_OHE_Germany', 'Geography_OHE_Spain', 'Tenure_OHE_0.0',
       'Tenure_OHE_1.0', 'Tenure_OHE_2.0', 'Tenure_OHE_3.0', 'Tenure_OHE_4.0',
       'Tenure_OHE_5.0', 'Tenure_OHE_6.0', 'Tenure_OHE_7.0', 'Tenure_OHE_8.0',
       'Tenure_OHE_9.0', 'Tenure_OHE_10.0', 'NumOfProducts_OHE_1.0',
       'NumOfProducts_OHE_2.0', 'NumOfProducts_OHE_3.0', 'Gender_OHE_Female',
       'Gender_OHE_Male', 'log_CreditScore', 'log_Age', 'log_Balance',
       'log_EstimatedSalary'],
      dtype='object')

In [70]:
list(Xtrain_trans) == list(Xtest_trans )

True

In [71]:
Xtrain_trans

,HasCrCard,IsActiveMember,Geography_OHE_France,Geography_OHE_Germany,Geography_OHE_Spain,Tenure_OHE_0.0,Tenure_OHE_1.0,Tenure_OHE_2.0,Tenure_OHE_3.0,Tenure_OHE_4.0,...,Tenure_OHE_10.0,NumOfProducts_OHE_1.0,NumOfProducts_OHE_2.0,NumOfProducts_OHE_3.0,Gender_OHE_Female,Gender_OHE_Male,log_CreditScore,log_Age,log_Balance,log_EstimatedSalary
116180,1.0,1.0,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,6.621406,3.871201,11.706658,10.905490
152256,0.0,1.0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,6.498282,3.713572,0.000000,10.133263
124302,1.0,1.0,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,6.565265,3.761200,0.000000,11.927319
79682,1.0,1.0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,6.527958,3.496508,0.000000,11.288783
127888,0.0,0.0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,1,6.655440,3.610918,0.000000,11.065229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144921,1.0,0.0,1,0,0,0,0,1,0,0,...,0,0,1,0,0,1,6.356108,3.688879,0.000000,11.305412
41674,1.0,0.0,0,0,1,0,0,0,1,0,...,0,0,1,0,1,0,6.453625,3.496508,0.000000,11.392572
73618,1.0,0.0,1,0,0,0,1,0,0,0,...,0,1,0,0,1,0,6.736967,3.663562,11.862085,11.552896
122092,0.0,1.0,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,6.383507,3.433987,11.437260,12.057366


In [ ]:
cols = cont_cols
cols

In [72]:
def generate_features(col1, col2):
        temp_df = pd.DataFrame()  # Temporary dataframe to store the generated columns
        temp_df_test = pd.DataFrame()  # Temporary dataframe for test data

        operations = [('*', lambda x, y: x * y),
                      ('/', lambda x, y: x / (y + 1e-5)),
                      ('/', lambda x, y: y / (x + 1e-5)),
                      ('-', lambda x, y: x - y),
                      ('+', lambda x, y: x + y)]

        for op_name, operation in operations:
            new_col_name = f"{col1}{op_name}{col2}"
            temp_df[new_col_name] = operation(Xtrain[col1], Xtrain[col2])
            temp_df_test[new_col_name] = operation(Xtest[col1], Xtest[col2])

        return temp_df, temp_df_test

In [73]:
generate_features('CreditScore', 'Balance')

(        CreditScore*Balance  CreditScore/Balance  CreditScore-Balance  \
 116180         9.103211e+07           161.834865           -120626.15   
 152256         0.000000e+00             0.000000               663.00   
 124302         0.000000e+00             0.000000               709.00   
 79682          0.000000e+00             0.000000               683.00   
 127888         0.000000e+00             0.000000               776.00   
 ...                     ...                  ...                  ...   
 144921         0.000000e+00             0.000000               575.00   
 41674          0.000000e+00             0.000000               634.00   
 73618          1.193843e+08           168.392599           -140944.57   
 122092         5.479259e+07           156.872518            -92120.66   
 7704           9.053534e+07           231.770460           -144231.54   
 
         CreditScore+Balance  
 116180            122126.15  
 152256               663.00  
 124302          

In [74]:
def better_features(Xtrain, Xtest, Ytrain, cols, best_score):
    new_cols = []
    skf = KFold(n_splits=5, shuffle=True, random_state=42)  # Stratified k-fold object
    
    def generate_features(col1, col2):
        temp_df = pd.DataFrame()  # Temporary dataframe to store the generated columns
        temp_df_test = pd.DataFrame()  # Temporary dataframe for test data

        operations = [('*', lambda x, y: x * y),
                      ('/', lambda x, y: x / (y + 1e-5)),
                      ('/', lambda x, y: y / (x + 1e-5)),
                      ('-', lambda x, y: x - y),
                      ('+', lambda x, y: x + y)]
        for op_name, operation in operations:
            new_col_name = f"{col1}{op_name}{col2}"
            temp_df[new_col_name] = operation(Xtrain[col1], Xtrain[col2])
            temp_df_test[new_col_name] = operation(Xtest[col1], Xtest[col2])

        return temp_df, temp_df_test
    # Initialisez une liste pour stocker les résultats
    results = []
    # Boucle sur les colonnes avec tqdm pour afficher une barre de progression
    for i in tqdm(range(len(cols)), desc='Generating Features'):
        col1 = cols[i]
        temp_cols = cols[i+1:]
        # Appel à generate_features pour chaque paire de colonnes
        for col2 in temp_cols:
            temp_df, temp_df_test = generate_features(col1, col2)
            results.append((temp_df, temp_df_test))
    for temp_df, temp_df_test in results:
        for column in temp_df.columns:
            scores = []
            for train_index, val_index in skf.split(Xtrain, Ytrain):
                X_train, X_val = temp_df[column].iloc[train_index].values.reshape(-1, 1), temp_df[column].iloc[val_index].values.reshape(-1, 1)
                y_train, y_val = Ytrain.astype(int).iloc[train_index], Ytrain.astype(int).iloc[val_index]   
                model = HistGradientBoostingClassifier(max_iter=300, learning_rate=0.02, max_depth=6, random_state=42)
                model.fit(X_train, y_train)  
                y_pred = model.predict_proba(X_val)[:,1]
                score = roc_auc_score( y_val, y_pred)
                scores.append(score)    
            mean_score = np.mean(scores)
            if mean_score > best_score:
                new_col_name = temp_df[column].name
                corr_with_other_cols = Xtrain.corrwith(temp_df[column])
                if (corr_with_other_cols.abs().max() < 0.9 or best_score) and corr_with_other_cols.abs().max() != 1:
                    Xtrain[new_col_name] = temp_df[column]
                    Xtest[new_col_name] = temp_df_test[column]
                    new_cols.append(new_col_name)
                    print(f"Added column '{new_col_name}' with ROC AUC Score: {mean_score:.4f} & Correlation {corr_with_other_cols.abs().max():.4f}")
    return  Xtrain,  Xtest, new_cols

In [75]:
Xtrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132027 entries, 116180 to 7704
Data columns (total 10 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   CreditScore      132027 non-null  float64
 1   Geography        132027 non-null  object 
 2   Gender           132027 non-null  object 
 3   Age              132027 non-null  float64
 4   Tenure           132027 non-null  float64
 5   Balance          132027 non-null  float64
 6   NumOfProducts    132027 non-null  float64
 7   HasCrCard        132027 non-null  float64
 8   IsActiveMember   132027 non-null  float64
 9   EstimatedSalary  132027 non-null  float64
dtypes: float64(8), object(2)
memory usage: 11.1+ MB


In [76]:
Xtrain_bf, Xtest_bf, new_cols = better_features(Xtrain,Xtest, Ytrain, cont_cols, best_score = 0.60)

Generating Features: 100%|███████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 31.25it/s]


Added column 'CreditScore*Age' with ROC AUC Score: 0.7093 & Correlation 0.8787
Added column 'CreditScore/Age' with ROC AUC Score: 0.7265 & Correlation 0.8699
Added column 'CreditScore-Balance' with ROC AUC Score: 0.6058 & Correlation 1.0000
Added column 'CreditScore+Balance' with ROC AUC Score: 0.6059 & Correlation 1.0000
Added column 'Age*Balance' with ROC AUC Score: 0.6308 & Correlation 0.9543
Added column 'Age/Balance' with ROC AUC Score: 0.6301 & Correlation 0.9559
Added column 'Age-Balance' with ROC AUC Score: 0.7143 & Correlation 1.0000
Added column 'Age+Balance' with ROC AUC Score: 0.7139 & Correlation 1.0000
Added column 'Age*EstimatedSalary' with ROC AUC Score: 0.6195 & Correlation 0.8695


In [77]:
list(Xtrain_trans) == list(Xtest_trans)

True

In [78]:
print(Xtrain_bf.columns,Xtest_bf.columns)

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'CreditScore*Age', 'CreditScore/Age', 'CreditScore-Balance',
       'CreditScore+Balance', 'Age*Balance', 'Age/Balance', 'Age-Balance',
       'Age+Balance', 'Age*EstimatedSalary'],
      dtype='object') Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'CreditScore*Age', 'CreditScore/Age', 'CreditScore-Balance',
       'CreditScore+Balance', 'Age*Balance', 'Age/Balance', 'Age-Balance',
       'Age+Balance', 'Age*EstimatedSalary'],
      dtype='object')


In [79]:
Xtrain_bf_merg = Xtrain_bf.drop(['Geography', 'Gender', 'Tenure',
                                 'NumOfProducts', 'HasCrCard',
                                 'IsActiveMember'], axis = 1)

Xtest_bf_merg = Xtest_bf.drop(['Geography', 'Gender', 'Tenure',
                                 'NumOfProducts', 'HasCrCard',
                                 'IsActiveMember'], axis = 1)


In [80]:
print(Xtrain_bf.columns,Xtest_bf.columns)

Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'CreditScore*Age', 'CreditScore/Age', 'CreditScore-Balance',
       'CreditScore+Balance', 'Age*Balance', 'Age/Balance', 'Age-Balance',
       'Age+Balance', 'Age*EstimatedSalary'],
      dtype='object') Index(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
       'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary',
       'CreditScore*Age', 'CreditScore/Age', 'CreditScore-Balance',
       'CreditScore+Balance', 'Age*Balance', 'Age/Balance', 'Age-Balance',
       'Age+Balance', 'Age*EstimatedSalary'],
      dtype='object')


In [81]:
list(Xtrain_bf.columns) == list(Xtest_bf.columns)

True

In [82]:
Xtest_trans

,HasCrCard,IsActiveMember,Geography_OHE_France,Geography_OHE_Germany,Geography_OHE_Spain,Tenure_OHE_0.0,Tenure_OHE_1.0,Tenure_OHE_2.0,Tenure_OHE_3.0,Tenure_OHE_4.0,...,Tenure_OHE_10.0,NumOfProducts_OHE_1.0,NumOfProducts_OHE_2.0,NumOfProducts_OHE_3.0,Gender_OHE_Female,Gender_OHE_Male,log_CreditScore,log_Age,log_Balance,log_EstimatedSalary
59556,1.0,0.0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,6.246107,3.555348,0.000000,10.204465
18893,1.0,1.0,1,0,0,0,0,0,0,1,...,0,0,1,0,1,0,6.612041,3.737670,0.000000,12.159568
38472,1.0,0.0,1,0,0,0,0,0,0,1,...,0,0,1,0,0,1,6.444131,3.367296,0.000000,10.811646
53391,1.0,0.0,0,0,1,0,0,0,0,0,...,0,1,0,0,1,0,6.135565,3.583519,0.000000,11.804216
108842,1.0,1.0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,6.204558,3.583519,11.657634,9.640807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30644,0.0,0.0,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,6.375025,3.367296,11.547556,11.480753
115675,1.0,0.0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,1,6.248043,3.496508,0.000000,11.395167
4440,0.0,1.0,1,0,0,0,0,0,0,0,...,0,1,0,0,1,0,6.188264,3.583519,11.490888,11.700129
43916,1.0,0.0,0,0,1,0,1,0,0,0,...,0,1,0,0,0,1,6.612041,3.610918,0.000000,12.181746


In [83]:
Xtrain_final = pd.concat([Xtrain_bf_merg,Xtrain_trans], axis = 1)
Xtest_final = pd.concat([Xtest_bf_merg,Xtest_trans], axis = 1)
Xtrain_final

,CreditScore,Age,Balance,EstimatedSalary,CreditScore*Age,CreditScore/Age,CreditScore-Balance,CreditScore+Balance,Age*Balance,Age/Balance,...,Tenure_OHE_10.0,NumOfProducts_OHE_1.0,NumOfProducts_OHE_2.0,NumOfProducts_OHE_3.0,Gender_OHE_Female,Gender_OHE_Male,log_CreditScore,log_Age,log_Balance,log_EstimatedSalary
116180,750.0,47.0,121376.15,54473.60,35250.0,0.062667,-120626.15,122126.15,5704679.05,2582.470727,...,0,0,1,0,1,0,6.621406,3.871201,11.706658,10.905490
152256,663.0,40.0,0.00,25165.36,26520.0,0.060332,663.00,663.00,0.00,0.000000,...,0,0,1,0,0,1,6.498282,3.713572,0.000000,10.133263
124302,709.0,42.0,0.00,151344.23,29778.0,0.059238,709.00,709.00,0.00,0.000000,...,0,0,1,0,1,0,6.565265,3.761200,0.000000,11.927319
79682,683.0,32.0,0.00,79919.13,21856.0,0.046852,683.00,683.00,0.00,0.000000,...,0,0,1,0,0,1,6.527958,3.496508,0.000000,11.288783
127888,776.0,36.0,0.00,63908.86,27936.0,0.046392,776.00,776.00,0.00,0.000000,...,0,0,1,0,0,1,6.655440,3.610918,0.000000,11.065229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144921,575.0,39.0,0.00,81259.25,22425.0,0.067826,575.00,575.00,0.00,0.000000,...,0,0,1,0,0,1,6.356108,3.688879,0.000000,11.305412
41674,634.0,32.0,0.00,88659.67,20288.0,0.050473,634.00,634.00,0.00,0.000000,...,0,0,1,0,1,0,6.453625,3.496508,0.000000,11.392572
73618,842.0,38.0,141786.57,104077.01,31996.0,0.045131,-140944.57,142628.57,5387889.66,3731.224544,...,0,1,0,0,1,0,6.736967,3.663562,11.862085,11.552896
122092,591.0,30.0,92711.66,172363.46,17730.0,0.050761,-92120.66,93302.66,2781349.80,3090.387637,...,0,1,0,0,1,0,6.383507,3.433987,11.437260,12.057366


In [84]:
Xtest_final

,CreditScore,Age,Balance,EstimatedSalary,CreditScore*Age,CreditScore/Age,CreditScore-Balance,CreditScore+Balance,Age*Balance,Age/Balance,...,Tenure_OHE_10.0,NumOfProducts_OHE_1.0,NumOfProducts_OHE_2.0,NumOfProducts_OHE_3.0,Gender_OHE_Female,Gender_OHE_Male,log_CreditScore,log_Age,log_Balance,log_EstimatedSalary
59556,515.0,34.0,0.00,27022.57,17510.0,0.066019,515.00,515.00,0.00,0.000000,...,0,0,1,0,0,1,6.246107,3.555348,0.000000,10.204465
18893,743.0,41.0,0.00,190911.02,30463.0,0.055182,743.00,743.00,0.00,0.000000,...,0,0,1,0,1,0,6.612041,3.737670,0.000000,12.159568
38472,628.0,28.0,0.00,49594.02,17584.0,0.044586,628.00,628.00,0.00,0.000000,...,0,0,1,0,0,1,6.444131,3.367296,0.000000,10.811646
53391,461.0,35.0,0.00,133814.28,16135.0,0.075922,461.00,461.00,0.00,0.000000,...,0,1,0,0,1,0,6.135565,3.583519,0.000000,11.804216
108842,494.0,35.0,115569.21,15378.75,17290.0,0.070850,-115075.21,116063.21,4044922.35,3301.976485,...,0,0,1,0,0,1,6.204558,3.583519,11.657634,9.640807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30644,586.0,28.0,103522.75,96833.00,16408.0,0.047782,-102936.75,104108.75,2898637.00,3697.239751,...,0,1,0,0,1,0,6.375025,3.367296,11.547556,11.480753
115675,516.0,32.0,0.00,88890.05,16512.0,0.062016,516.00,516.00,0.00,0.000000,...,0,0,1,0,0,1,6.248043,3.496508,0.000000,11.395167
4440,486.0,35.0,97819.36,120586.32,17010.0,0.072016,-97333.36,98305.36,3423677.60,2794.838059,...,0,1,0,0,1,0,6.188264,3.583519,11.490888,11.700129
43916,743.0,36.0,0.00,195192.40,26748.0,0.048452,743.00,743.00,0.00,0.000000,...,0,1,0,0,0,1,6.612041,3.610918,0.000000,12.181746


In [85]:
list(Xtrain_final) ==list(Xtest_final)

True

In [86]:
Xtrain_final.columns

Index(['CreditScore', 'Age', 'Balance', 'EstimatedSalary', 'CreditScore*Age',
       'CreditScore/Age', 'CreditScore-Balance', 'CreditScore+Balance',
       'Age*Balance', 'Age/Balance', 'Age-Balance', 'Age+Balance',
       'Age*EstimatedSalary', 'HasCrCard', 'IsActiveMember',
       'Geography_OHE_France', 'Geography_OHE_Germany', 'Geography_OHE_Spain',
       'Tenure_OHE_0.0', 'Tenure_OHE_1.0', 'Tenure_OHE_2.0', 'Tenure_OHE_3.0',
       'Tenure_OHE_4.0', 'Tenure_OHE_5.0', 'Tenure_OHE_6.0', 'Tenure_OHE_7.0',
       'Tenure_OHE_8.0', 'Tenure_OHE_9.0', 'Tenure_OHE_10.0',
       'NumOfProducts_OHE_1.0', 'NumOfProducts_OHE_2.0',
       'NumOfProducts_OHE_3.0', 'Gender_OHE_Female', 'Gender_OHE_Male',
       'log_CreditScore', 'log_Age', 'log_Balance', 'log_EstimatedSalary'],
      dtype='object')

In [87]:
Xtest_final.columns

Index(['CreditScore', 'Age', 'Balance', 'EstimatedSalary', 'CreditScore*Age',
       'CreditScore/Age', 'CreditScore-Balance', 'CreditScore+Balance',
       'Age*Balance', 'Age/Balance', 'Age-Balance', 'Age+Balance',
       'Age*EstimatedSalary', 'HasCrCard', 'IsActiveMember',
       'Geography_OHE_France', 'Geography_OHE_Germany', 'Geography_OHE_Spain',
       'Tenure_OHE_0.0', 'Tenure_OHE_1.0', 'Tenure_OHE_2.0', 'Tenure_OHE_3.0',
       'Tenure_OHE_4.0', 'Tenure_OHE_5.0', 'Tenure_OHE_6.0', 'Tenure_OHE_7.0',
       'Tenure_OHE_8.0', 'Tenure_OHE_9.0', 'Tenure_OHE_10.0',
       'NumOfProducts_OHE_1.0', 'NumOfProducts_OHE_2.0',
       'NumOfProducts_OHE_3.0', 'Gender_OHE_Female', 'Gender_OHE_Male',
       'log_CreditScore', 'log_Age', 'log_Balance', 'log_EstimatedSalary'],
      dtype='object')

In [88]:
def post_processor(train, test):
    # Identifie les colonnes en double dans l'ensemble de données d'entraînement
    duplicate_columns = train.loc[:, train.columns.duplicated()].columns
    
    # Supprime les colonnes en double de l'ensemble de données d'entraînement et de test
    train = train.drop(columns=duplicate_columns)
    test = test.drop(columns=duplicate_columns)
    return train, test
             
#train_cop, test_cop=   post_processor(train_scaled, test_scaled)

In [89]:
Xtrain_final, Xtest_final = post_processor(Xtrain_final, Xtest_final)
Xtrain_final

,CreditScore,Age,Balance,EstimatedSalary,CreditScore*Age,CreditScore/Age,CreditScore-Balance,CreditScore+Balance,Age*Balance,Age/Balance,...,Tenure_OHE_10.0,NumOfProducts_OHE_1.0,NumOfProducts_OHE_2.0,NumOfProducts_OHE_3.0,Gender_OHE_Female,Gender_OHE_Male,log_CreditScore,log_Age,log_Balance,log_EstimatedSalary
116180,750.0,47.0,121376.15,54473.60,35250.0,0.062667,-120626.15,122126.15,5704679.05,2582.470727,...,0,0,1,0,1,0,6.621406,3.871201,11.706658,10.905490
152256,663.0,40.0,0.00,25165.36,26520.0,0.060332,663.00,663.00,0.00,0.000000,...,0,0,1,0,0,1,6.498282,3.713572,0.000000,10.133263
124302,709.0,42.0,0.00,151344.23,29778.0,0.059238,709.00,709.00,0.00,0.000000,...,0,0,1,0,1,0,6.565265,3.761200,0.000000,11.927319
79682,683.0,32.0,0.00,79919.13,21856.0,0.046852,683.00,683.00,0.00,0.000000,...,0,0,1,0,0,1,6.527958,3.496508,0.000000,11.288783
127888,776.0,36.0,0.00,63908.86,27936.0,0.046392,776.00,776.00,0.00,0.000000,...,0,0,1,0,0,1,6.655440,3.610918,0.000000,11.065229
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144921,575.0,39.0,0.00,81259.25,22425.0,0.067826,575.00,575.00,0.00,0.000000,...,0,0,1,0,0,1,6.356108,3.688879,0.000000,11.305412
41674,634.0,32.0,0.00,88659.67,20288.0,0.050473,634.00,634.00,0.00,0.000000,...,0,0,1,0,1,0,6.453625,3.496508,0.000000,11.392572
73618,842.0,38.0,141786.57,104077.01,31996.0,0.045131,-140944.57,142628.57,5387889.66,3731.224544,...,0,1,0,0,1,0,6.736967,3.663562,11.862085,11.552896
122092,591.0,30.0,92711.66,172363.46,17730.0,0.050761,-92120.66,93302.66,2781349.80,3090.387637,...,0,1,0,0,1,0,6.383507,3.433987,11.437260,12.057366


In [90]:
len(Xtrain_final.columns) == len(Xtest_final.columns)

True

In [91]:
Xtrain_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132027 entries, 116180 to 7704
Data columns (total 38 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CreditScore            132027 non-null  float64
 1   Age                    132027 non-null  float64
 2   Balance                132027 non-null  float64
 3   EstimatedSalary        132027 non-null  float64
 4   CreditScore*Age        132027 non-null  float64
 5   CreditScore/Age        132027 non-null  float64
 6   CreditScore-Balance    132027 non-null  float64
 7   CreditScore+Balance    132027 non-null  float64
 8   Age*Balance            132027 non-null  float64
 9   Age/Balance            132027 non-null  float64
 10  Age-Balance            132027 non-null  float64
 11  Age+Balance            132027 non-null  float64
 12  Age*EstimatedSalary    132027 non-null  float64
 13  HasCrCard              132027 non-null  float64
 14  IsActiveMember         132027 non

In [92]:
# Find columns with uint8 dtype
cols_to_conv = list(Xtrain_final.select_dtypes(include='uint8').columns) + ['HasCrCard', 'IsActiveMember']
# Convert uint8 columns to object type
Xtrain_final[cols_to_conv] = Xtrain_final[cols_to_conv].astype('object')
Xtrain_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132027 entries, 116180 to 7704
Data columns (total 38 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CreditScore            132027 non-null  float64
 1   Age                    132027 non-null  float64
 2   Balance                132027 non-null  float64
 3   EstimatedSalary        132027 non-null  float64
 4   CreditScore*Age        132027 non-null  float64
 5   CreditScore/Age        132027 non-null  float64
 6   CreditScore-Balance    132027 non-null  float64
 7   CreditScore+Balance    132027 non-null  float64
 8   Age*Balance            132027 non-null  float64
 9   Age/Balance            132027 non-null  float64
 10  Age-Balance            132027 non-null  float64
 11  Age+Balance            132027 non-null  float64
 12  Age*EstimatedSalary    132027 non-null  float64
 13  HasCrCard              132027 non-null  object 
 14  IsActiveMember         132027 non

In [93]:
Ytrain = Ytrain.astype("object")
Ytest = Ytest.astype("object")
Ytrain

116180    0
152256    0
124302    0
79682     0
127888    0
         ..
144921    0
41674     0
73618     0
122092    0
7704      0
Name: Exited, Length: 132027, dtype: object

In [94]:
def compute_correlation(x, y):
    """
    Compute correlation between two variables.

    Parameters:
    x (pd.Series): First variable.
    y (pd.Series): Second variable.

    Returns:
    float: Correlation value.
    """
    if x.dtype == 'object' and y.dtype == 'object':
        # Both variables are categorical
        contingency_table = pd.crosstab(x, y)
        _, p, _, _ = chi2_contingency(contingency_table)
        correlation = np.sqrt(p)
    elif x.dtype == 'object' or y.dtype == 'object':
        # One variable is categorical, one is continuous
        if x.dtype == 'object':
            categorical = x
            continuous = y
        else:
            categorical = y
            continuous = x
        point_biserial_corr, _ = pointbiserialr(continuous, categorical)
        correlation = np.abs(point_biserial_corr)
    else:
        # Both variables are continuous
        correlation, _ = pearsonr(x, y)
        
    return correlation

In [95]:
def remove_correlated_features(Xtrain, Ytrain, Xtest, Ytest, threshold=0.9):
    """
    Remove highly correlated features while considering their influence on the target variable and between themselves.

    Parameters:
    df (pd.DataFrame): DataFrame containing features and target variable.
    target_col (str): Name of the target variable column.
    threshold (float): Correlation threshold above which features are considered highly correlated. Default is 0.9.

    Returns:
    pd.DataFrame: DataFrame with correlated features removed.
    """
    
    X = pd.concat([Xtrain, Xtest], ignore_index=True)
    Y = pd.concat([Ytrain, Ytest], ignore_index=True)
    train_indx = Xtrain.index
    test_indx  = Xtest.index

    # Compute correlation between features and target variable 
    feature_target_corr = {}
    for feature in X.columns:
        correlation = compute_correlation(X[feature], Y)
        feature_target_corr[feature] = correlation

    # Compute correlation between features themselves
    feature_feature_corr = X.corr().abs().stack().reset_index().sort_values(0, ascending=False)
    feature_feature_corr.rename(columns={0: 'correlation'}, inplace=True)
    feature_feature_corr = feature_feature_corr[feature_feature_corr['level_0'] != feature_feature_corr['level_1']]

    # Find highly correlated features
    highly_corr_features = set()
    for index, row in feature_feature_corr.iterrows():
        feature1, feature2 = row['level_0'], row['level_1']
        if abs(row['correlation']) > threshold:
            highly_corr_features.add(feature1)
            highly_corr_features.add(feature2)

    # Combine correlated features with high target correlation
    correlated_features = set(highly_corr_features) | set(feature for feature, corr in feature_target_corr.items() if abs(corr) > threshold)

    # Remove highly correlated features from DataFrame
    
    X_filtered = X.drop(columns=correlated_features)
    Xtrain_filtered = X_filtered.loc[train_indx]
    Xtest_filtered  = X_filtered.loc[test_indx]
    
    return Xtrain_filtered, Xtest_filtered 

In [96]:
Xtrain_final.isnull().sum()

CreditScore              0
Age                      0
Balance                  0
EstimatedSalary          0
CreditScore*Age          0
CreditScore/Age          0
CreditScore-Balance      0
CreditScore+Balance      0
Age*Balance              0
Age/Balance              0
Age-Balance              0
Age+Balance              0
Age*EstimatedSalary      0
HasCrCard                0
IsActiveMember           0
Geography_OHE_France     0
Geography_OHE_Germany    0
Geography_OHE_Spain      0
Tenure_OHE_0.0           0
Tenure_OHE_1.0           0
Tenure_OHE_2.0           0
Tenure_OHE_3.0           0
Tenure_OHE_4.0           0
Tenure_OHE_5.0           0
Tenure_OHE_6.0           0
Tenure_OHE_7.0           0
Tenure_OHE_8.0           0
Tenure_OHE_9.0           0
Tenure_OHE_10.0          0
NumOfProducts_OHE_1.0    0
NumOfProducts_OHE_2.0    0
NumOfProducts_OHE_3.0    0
Gender_OHE_Female        0
Gender_OHE_Male          0
log_CreditScore          0
log_Age                  0
log_Balance              0
l

In [97]:
Ytrain.isnull().sum()

0

In [98]:
Ytest

59556     0
18893     0
38472     0
53391     0
108842    0
         ..
30644     0
115675    0
4440      0
43916     0
128435    0
Name: Exited, Length: 33007, dtype: object

In [99]:
Xtest_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33007 entries, 59556 to 128435
Data columns (total 38 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   CreditScore            33007 non-null  float64
 1   Age                    33007 non-null  float64
 2   Balance                33007 non-null  float64
 3   EstimatedSalary        33007 non-null  float64
 4   CreditScore*Age        33007 non-null  float64
 5   CreditScore/Age        33007 non-null  float64
 6   CreditScore-Balance    33007 non-null  float64
 7   CreditScore+Balance    33007 non-null  float64
 8   Age*Balance            33007 non-null  float64
 9   Age/Balance            33007 non-null  float64
 10  Age-Balance            33007 non-null  float64
 11  Age+Balance            33007 non-null  float64
 12  Age*EstimatedSalary    33007 non-null  float64
 13  HasCrCard              33007 non-null  float64
 14  IsActiveMember         33007 non-null  float64
 1

In [100]:
Xtrain_final,Xtest_final = remove_correlated_features(Xtrain_final, Ytrain, Xtest_final, Ytest, threshold=0.9)


In [101]:
Xtest_final

,EstimatedSalary,CreditScore*Age,CreditScore/Age,Age*EstimatedSalary,HasCrCard,IsActiveMember,Geography_OHE_France,Geography_OHE_Germany,Geography_OHE_Spain,Tenure_OHE_0.0,...,Tenure_OHE_5.0,Tenure_OHE_6.0,Tenure_OHE_7.0,Tenure_OHE_8.0,NumOfProducts_OHE_1.0,NumOfProducts_OHE_2.0,NumOfProducts_OHE_3.0,Gender_OHE_Female,Gender_OHE_Male,log_EstimatedSalary
59556,106534.15,34320.0,0.067133,5113639.20,1.0,1.0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,11.576230
18893,150507.43,34638.0,0.061089,6923341.78,1.0,0.0,1,0,0,0,...,0,0,1,0,0,1,0,0,1,11.921774
38472,94732.86,22572.0,0.048246,3126184.38,1.0,0.0,0,0,1,0,...,0,0,0,1,0,1,0,1,0,11.458827
53391,177028.00,21054.0,0.039945,5133812.00,1.0,0.0,0,1,0,0,...,0,0,0,0,0,1,0,0,1,12.084069
108842,79722.11,24528.0,0.071918,3348328.62,1.0,1.0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,11.286315
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30644,4176.17,36456.0,0.086022,233865.52,1.0,0.0,1,0,0,0,...,0,0,0,0,1,0,0,1,0,8.337389
115675,57553.62,24776.0,0.058282,2187037.56,1.0,0.0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,10.960490
4440,176984.34,19278.0,0.037815,4778577.18,1.0,0.0,1,0,0,0,...,0,0,0,0,0,1,0,0,1,12.083822
43916,167984.61,31038.0,0.056834,7055353.62,1.0,1.0,1,0,0,0,...,1,0,0,0,0,1,0,0,1,12.031634


In [102]:
Xtrain_final

,EstimatedSalary,CreditScore*Age,CreditScore/Age,Age*EstimatedSalary,HasCrCard,IsActiveMember,Geography_OHE_France,Geography_OHE_Germany,Geography_OHE_Spain,Tenure_OHE_0.0,...,Tenure_OHE_5.0,Tenure_OHE_6.0,Tenure_OHE_7.0,Tenure_OHE_8.0,NumOfProducts_OHE_1.0,NumOfProducts_OHE_2.0,NumOfProducts_OHE_3.0,Gender_OHE_Female,Gender_OHE_Male,log_EstimatedSalary
116180,160129.65,26233.0,0.052186,5924797.05,1.0,1.0,1,0,0,0,...,1,0,0,0,1,0,0,0,1,11.983745
152256,143783.98,33894.0,0.052045,6038927.16,1.0,0.0,0,1,0,0,...,0,0,0,1,1,0,0,0,1,11.876074
124302,74771.22,28577.0,0.058824,3065620.02,1.0,1.0,0,1,0,0,...,0,0,0,0,1,0,0,0,1,11.222202
79682,37013.29,25174.0,0.066775,1517544.89,0.0,1.0,1,0,0,0,...,0,0,1,0,0,1,0,1,0,10.519059
127888,145765.83,36034.0,0.051313,6267930.69,1.0,1.0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,11.889764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144921,164394.65,21168.0,0.037037,4603050.20,1.0,0.0,1,0,0,0,...,0,1,0,0,1,0,0,1,0,12.010031
41674,7861.68,32715.0,0.061898,353775.60,1.0,1.0,0,0,1,0,...,0,0,0,0,1,0,0,0,1,8.969883
73618,183569.25,19460.0,0.062950,6424923.75,1.0,1.0,1,0,0,0,...,0,0,0,0,0,1,0,1,0,12.120353
122092,125451.01,16929.0,0.043062,3387177.27,1.0,1.0,0,0,1,0,...,0,0,0,0,0,1,0,1,0,11.739679


In [103]:
Xtrain_final.columns

Index(['EstimatedSalary', 'CreditScore*Age', 'CreditScore/Age',
       'Age*EstimatedSalary', 'HasCrCard', 'IsActiveMember',
       'Geography_OHE_France', 'Geography_OHE_Germany', 'Geography_OHE_Spain',
       'Tenure_OHE_0.0', 'Tenure_OHE_1.0', 'Tenure_OHE_2.0', 'Tenure_OHE_3.0',
       'Tenure_OHE_4.0', 'Tenure_OHE_5.0', 'Tenure_OHE_6.0', 'Tenure_OHE_7.0',
       'Tenure_OHE_8.0', 'NumOfProducts_OHE_1.0', 'NumOfProducts_OHE_2.0',
       'NumOfProducts_OHE_3.0', 'Gender_OHE_Female', 'Gender_OHE_Male',
       'log_EstimatedSalary'],
      dtype='object')

In [104]:
cat_cols = Xtrain_final.select_dtypes(include = "object").columns
cat_cols 

Index(['HasCrCard', 'IsActiveMember', 'Geography_OHE_France',
       'Geography_OHE_Germany', 'Geography_OHE_Spain', 'Tenure_OHE_0.0',
       'Tenure_OHE_1.0', 'Tenure_OHE_2.0', 'Tenure_OHE_3.0', 'Tenure_OHE_4.0',
       'Tenure_OHE_5.0', 'Tenure_OHE_6.0', 'Tenure_OHE_7.0', 'Tenure_OHE_8.0',
       'NumOfProducts_OHE_1.0', 'NumOfProducts_OHE_2.0',
       'NumOfProducts_OHE_3.0', 'Gender_OHE_Female', 'Gender_OHE_Male'],
      dtype='object')

In [105]:
Xtrain_final[cat_cols].astype("int")

,HasCrCard,IsActiveMember,Geography_OHE_France,Geography_OHE_Germany,Geography_OHE_Spain,Tenure_OHE_0.0,Tenure_OHE_1.0,Tenure_OHE_2.0,Tenure_OHE_3.0,Tenure_OHE_4.0,Tenure_OHE_5.0,Tenure_OHE_6.0,Tenure_OHE_7.0,Tenure_OHE_8.0,NumOfProducts_OHE_1.0,NumOfProducts_OHE_2.0,NumOfProducts_OHE_3.0,Gender_OHE_Female,Gender_OHE_Male
116180,1,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1
152256,1,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1
124302,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1
79682,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0
127888,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144921,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0
41674,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1
73618,1,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0
122092,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0


In [106]:
Xtrain_final[cat_cols] = Xtrain_final[cat_cols].astype("int")
Xtrain_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132027 entries, 116180 to 7704
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   EstimatedSalary        132027 non-null  float64
 1   CreditScore*Age        132027 non-null  float64
 2   CreditScore/Age        132027 non-null  float64
 3   Age*EstimatedSalary    132027 non-null  float64
 4   HasCrCard              132027 non-null  int32  
 5   IsActiveMember         132027 non-null  int32  
 6   Geography_OHE_France   132027 non-null  int32  
 7   Geography_OHE_Germany  132027 non-null  int32  
 8   Geography_OHE_Spain    132027 non-null  int32  
 9   Tenure_OHE_0.0         132027 non-null  int32  
 10  Tenure_OHE_1.0         132027 non-null  int32  
 11  Tenure_OHE_2.0         132027 non-null  int32  
 12  Tenure_OHE_3.0         132027 non-null  int32  
 13  Tenure_OHE_4.0         132027 non-null  int32  
 14  Tenure_OHE_5.0         132027 non

In [107]:
Xtest_final[cat_cols] = Xtest_final[cat_cols].astype("int")
Xtest_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 33007 entries, 59556 to 128435
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   EstimatedSalary        33007 non-null  float64
 1   CreditScore*Age        33007 non-null  float64
 2   CreditScore/Age        33007 non-null  float64
 3   Age*EstimatedSalary    33007 non-null  float64
 4   HasCrCard              33007 non-null  int32  
 5   IsActiveMember         33007 non-null  int32  
 6   Geography_OHE_France   33007 non-null  int32  
 7   Geography_OHE_Germany  33007 non-null  int32  
 8   Geography_OHE_Spain    33007 non-null  int32  
 9   Tenure_OHE_0.0         33007 non-null  int32  
 10  Tenure_OHE_1.0         33007 non-null  int32  
 11  Tenure_OHE_2.0         33007 non-null  int32  
 12  Tenure_OHE_3.0         33007 non-null  int32  
 13  Tenure_OHE_4.0         33007 non-null  int32  
 14  Tenure_OHE_5.0         33007 non-null  int32  
 1

In [108]:
print(Xtrain_final.shape, Ytrain.shape)

(132027, 24) (132027,)


In [109]:
print(Xtest_final.shape, Ytest.shape)

(33007, 24) (33007,)


In [110]:
from sklearn.preprocessing import LabelEncoder

In [114]:
# Convertir les étiquettes de classe en entiers avec LabelEncoder
label_encoder = LabelEncoder()
Ytrain = label_encoder.fit_transform(Ytrain)
# Vérifier les classes uniques dans les étiquettes de classe
classes = label_encoder.classes_
print("Classes uniques:", classes)

Classes uniques: [0 1]


In [115]:
# Convertir les étiquettes de classe en entiers avec LabelEncoder
label_encoder = LabelEncoder()
Ytest = label_encoder.fit_transform(Ytest)
# Vérifier les classes uniques dans les étiquettes de classe
classes = label_encoder.classes_
print("Classes uniques:", classes)

Classes uniques: [0 1]


In [116]:
type(Ytrain)

numpy.ndarray

In [ ]:
print(X_train.shape, X_val.shape)

### Modele 

In [ ]:
from sklearn.ensemble import AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
# Initialiser les classificateurs
adaboost_clf = AdaBoostClassifier(random_state=42)
extratrees_clf = ExtraTreesClassifier(random_state=42)
gradientboost_clf = GradientBoostingClassifier(random_state=42)
randomforest_clf = RandomForestClassifier(random_state=42)

In [ ]:
# Définir les grilles de recherche pour les hyperparamètres
adaboost_params = {'n_estimators': [50, 100, 200]}
extratrees_params = {'n_estimators': [50, 100, 200]}
gradientboost_params = {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 0.5]}
randomforest_params = {'n_estimators': [50, 100, 200]}


In [ ]:
# Initialiser les objets de recherche sur grille
adaboost_grid = GridSearchCV(adaboost_clf, adaboost_params, cv=5, scoring='accuracy')
extratrees_grid = GridSearchCV(extratrees_clf, extratrees_params, cv=5, scoring='accuracy')
gradientboost_grid = GridSearchCV(gradientboost_clf, gradientboost_params, cv=5, scoring='accuracy')
randomforest_grid = GridSearchCV(randomforest_clf, randomforest_params, cv=5, scoring='accuracy')


In [ ]:
# Effectuer la recherche sur grille pour chaque classificateur
adaboost_grid.fit(Xtrain_final, Ytrain)
extratrees_grid.fit(Xtrain_final, Ytrain)
gradientboost_grid.fit(Xtrain_final, Ytrain)
randomforest_grid.fit(Xtrain_final, Ytrain)

In [ ]:
# Sélectionner les meilleurs modèles
best_adaboost_clf = adaboost_grid.best_estimator_
best_extratrees_clf = extratrees_grid.best_estimator_
best_gradientboost_clf = gradientboost_grid.best_estimator_
best_randomforest_clf = randomforest_grid.best_estimator_

In [ ]:
best_adaboost_clf

In [ ]:
# Faire des prédictions sur l'ensemble de test avec les meilleurs modèles
adaboost_test_pred = best_adaboost_clf.predict(Xtest_final)
extratrees_test_pred = best_extratrees_clf.predict(Xtest_final)
gradientboost_test_pred = best_gradientboost_clf.predict(Xtest_final)
randomforest_test_pred = best_randomforest_clf.predict(Xtest_final)

In [ ]:
# Évaluer la performance des meilleurs modèles sur l'ensemble de test
adaboost_test_accuracy = accuracy_score(Ytest, adaboost_test_pred)
extratrees_test_accuracy = accuracy_score(Ytest, extratrees_test_pred)
gradientboost_test_accuracy = accuracy_score(Ytest, gradientboost_test_pred)
randomforest_test_accuracy = accuracy_score(Ytest, randomforest_test_pred)


In [ ]:
print("AdaBoost Accuracy on test set:", adaboost_test_accuracy)
print("ExtraTrees Accuracy on test set:", extratrees_test_accuracy)
print("GradientBoost Accuracy on test set:", gradientboost_test_accuracy)
print("RandomForest Accuracy on test set:", randomforest_test_accuracy)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# Définir les classificateurs de base pour le stacking
base_classifiers = [
    ('adaboost', best_adaboost_clf),
    ('extratrees', best_extratrees_clf),
    ('gradientboost', best_gradientboost_clf),
    ('randomforest', best_randomforest_clf)
]

In [ ]:
# Initialiser le classificateur de stacking avec une régression logistique comme estimateur final
stacking_clf = StackingClassifier(estimators=base_classifiers, final_estimator=LogisticRegression())
# Entraîner le classificateur de stacking sur l'ensemble d'entraînement et de validation combiné
stacking_clf.fit(Xtrain_final, Ytrain)


In [ ]:
# Faire des prédictions sur l'ensemble de test avec le classificateur de stacking
stacking_test_pred = stacking_clf.predict(Xtest_final)

In [ ]:
# Évaluer la performance du classificateur de stacking sur l'ensemble de test
stacking_test_accuracy = accuracy_score(Ytest, stacking_test_pred)
print("Stacking Accuracy on test set:", stacking_test_accuracy)

In [ ]:
Ytrain.info()

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [ ]:
class ModelTrainer:
    def __init__(self, Xtrain, Ytrain, Xtest, Ytest):
        self.Xtrain = Xtrain
        self.Ytrain = Ytrain
        self.Xtest  = Xtest
        self.Ytest  = Ytest
        self.label_encoder = LabelEncoder()
        self.best_models = {}
        
    def encode_labels(self):
        """
        Encode les étiquettes de classe en entiers.
        """
        self.Ytrain = self.label_encoder.fit_transform(self.Ytrain)
        self.Ytest = self.label_encoder.transform(self.Ytest)
    
    def train_models(self):
        """
        Entraîne les modèles de base sur les données d'entraînement et trouve les meilleurs hyperparamètres.
        """
        # Initialiser les classificateurs
        adaboost_clf = AdaBoostClassifier(random_state=42)
        extratrees_clf = ExtraTreesClassifier(random_state=42)
        gradientboost_clf = GradientBoostingClassifier(random_state=42)
        randomforest_clf = RandomForestClassifier(random_state=42)

        # Définir les grilles de recherche pour les hyperparamètres
        adaboost_params = {'n_estimators': [50, 100]}
        extratrees_params = {'n_estimators': [50, 100]}
        gradientboost_params = {'n_estimators': [50, 100],
                                'learning_rate': [0.01, 0.1]}
        randomforest_params = {'n_estimators': [50, 100]}
        
        # Initialiser les objets de recherche sur grille
        adaboost_grid = GridSearchCV(adaboost_clf, adaboost_params, cv=3, scoring='accuracy')
        extratrees_grid = GridSearchCV(extratrees_clf, extratrees_params, cv=5, scoring='accuracy')
        gradientboost_grid = GridSearchCV(gradientboost_clf, gradientboost_params, cv=5, scoring='accuracy')
        randomforest_grid = GridSearchCV(randomforest_clf, randomforest_params, cv=5, scoring='accuracy')

        # Effectuer la recherche sur grille pour chaque classificateur
        adaboost_grid.fit(self.Xtrain, self.Ytrain)
        extratrees_grid.fit(self.Xtrain, self.Ytrain)
        gradientboost_grid.fit(self.Xtrain, self.Ytrain)
        randomforest_grid.fit(self.Xtrain, self.Ytrain)

        # Stocker les meilleurs modèles
        self.best_models['adaboost'] = adaboost_grid.best_estimator_
        self.best_models['extratrees'] = extratrees_grid.best_estimator_
        self.best_models['gradientboost'] = gradientboost_grid.best_estimator_
        self.best_models['randomforest'] = randomforest_grid.best_estimator_
        
    def evaluate_models(self):
        """
        Évalue les modèles de base sur l'ensemble de validation.
        """
        for name, model in self.best_models.items():
            test_pred = model.predict(self.Xtest)
            test_accuracy = accuracy_score(self.Ytest, test_pred)
            print(f"{name.capitalize()} Accuracy on validation set: {test_accuracy}")
            
    def train_stacking_model(self):
        """
        Entraîne le modèle de stacking en utilisant les modèles de base comme estimateurs.
        """
        base_classifiers = [(name, model) for name, model in self.best_models.items()]
        self.stacking_clf = StackingClassifier(estimators=base_classifiers, final_estimator=LogisticRegression())
        self.stacking_clf.fit(self.Xtrain, self.Ytrain)
        
    def evaluate_stacking_model(self):
        """
        Évalue le modèle de stacking sur l'ensemble de test.
        """
        stacking_pred = self.stacking_clf.predict(self.Xtest)
        stacking_accuracy = accuracy_score(self.Ytest, stacking_pred)
        print("Stacking Accuracy on test set:", stacking_accuracy)
        
    def run(self):
        """
        Exécute le processus complet d'encodage, d'entraînement, d'évaluation et de test.
        """
        self.encode_labels()
        self.train_models()
        self.evaluate_models()
# Exemple d'utilisation
model_trainer = ModelTrainer(Xtrain_final, Ytrain, Xtest_final, Ytest)
model_trainer.run()

In [ ]:
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

In [ ]:
class Splitter:
    def __init__(self, test_size=0.2, kfold=True, n_splits=5):
        self.test_size = test_size
        self.kfold = kfold
        self.n_splits = n_splits

    def split_data(self, X, y, random_state_list):
        if self.kfold:
            for random_state in random_state_list:
                kf = KFold(n_splits=self.n_splits, random_state=random_state, shuffle=True)
                for train_index, val_index in kf.split(X, y):
                    X_train, X_val = X.iloc[train_index], X.iloc[val_index]
                    y_train, y_val = y.iloc[train_index], y.iloc[val_index]
                    yield X_train, X_val, y_train, y_val



class Classifier:
    def __init__(self, n_estimators=100, device="cpu", random_state=0):
        self.n_estimators = n_estimators
        self.device = device
        self.random_state = random_state
        self.models = self._define_model()
        self.len_models = len(self.models)
        
    def _define_model(self):
        # Define base model parameters
        xgb_base_params = {
            'n_estimators': self.n_estimators,
            'learning_rate': 0.1,
            'max_depth': 4,
            'subsample': 0.8,
            'colsample_bytree': 0.1,
            'n_jobs': -1,
            'eval_metric': 'logloss',
            'objective': 'binary:logistic',
            'tree_method': 'hist',
            'verbosity': 0,
            'random_state': self.random_state,
        }
        # Define model variations with different hyperparameters
        xgb_params = [
            xgb_base_params,
            {**xgb_base_params, 'subsample': 0.5, 'max_depth': 9, 'learning_rate': 0.045, 'colsample_bytree': 0.3},
            {**xgb_base_params, 'subsample': 0.6, 'max_depth': 6, 'learning_rate': 0.02, 'colsample_bytree': 0.7},
            {**xgb_base_params, 'subsample': 0.5943421542786502, 'max_depth': 6, 'learning_rate': 0.109, 'colsample_bytree': 0.5595039093313848},
        ]
        # Other models with different parameters
        # Define other models similarly
        
        models = {
            'xgb{}'.format(i+1): xgb.XGBClassifier(**params) for i, params in enumerate(xgb_params)
            # Add other models here
        }
        return models
    
    def optimize_hyperparameters(self, X_train, y_train):
        # Define hyperparameter grids for grid search
        xgb_param_grid = {
            'learning_rate': [0.1, 0.05, 0.01],
            'max_depth': [3, 5, 7],
            'subsample': [0.5, 0.8, 1.0],
            'colsample_bytree': [0.3, 0.5, 0.8]
        }
        # Add other hyperparameter grids for other models
        
        # Perform grid search for each model
        for name, model in self.models.items():
            grid_search = GridSearchCV(model, xgb_param_grid, cv=5, scoring='roc_auc', n_jobs=-1)
            grid_search.fit(X_train, y_train)
            self.models[name] = grid_search.best_estimator_
    
    def cross_validate(self, X_train, y_train):
        cv_scores = {}
        # Perform cross-validation for each model
        for name, model in self.models.items():
            scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc', n_jobs=-1)
            cv_scores[name] = scores.mean()
        return cv_scores
    
    def fit(self, X_train, y_train):
        # Fit each model on the training data
        for model in self.models.values():
            model.fit(X_train, y_train)
    
    def predict(self, X_test):
        # Predict using each model and return predictions
        predictions = {}
        for name, model in self.models.items():
            predictions[name] = model.predict(X_test)
        return predictions

In [ ]:
from sklearn.metrics import roc_auc_score, log_loss
import optuna
from optuna.samplers import CmaEsSampler
from optuna.pruners import HyperbandPruner
from functools import partial
import numpy as np

class Classifier:
    def __init__(self, model_type='xgb', n_estimators=100, random_state=0):
        self.model_type = model_type
        self.n_estimators = n_estimators
        self.random_state = random_state
        self.models = None
        self.feature_importances = None
        
    def optimize_hyperparameters(self, X_train, y_train):
        # Méthode d'optimisation des hyperparamètres spécifique à chaque type de modèle
        if self.model_type == 'xgb':
            # Optimisation des hyperparamètres pour XGBoost
            # Exemple : utiliser Optuna ou une autre méthode d'optimisation
            
            # Ceci est un exemple simplifié, vous devez implémenter l'optimisation réelle
            self.models = [XGBClassifier(n_estimators=self.n_estimators, random_state=self.random_state).fit(X_train, y_train)]
            
    def cross_validate(self, X_train, y_train):
        # Validation croisée spécifique à chaque type de modèle
        if self.model_type == 'xgb':
            # Validation croisée pour XGBoost
            # Exemple : utiliser la fonction cross_val_score de scikit-learn
            pass
            
    def fit(self, X_train, y_train):
        # Entraînement du modèle spécifique à chaque type de modèle
        if self.model_type == 'xgb':
            # Entraînement de XGBoost
            self.models = [XGBClassifier(n_estimators=self.n_estimators, random_state=self.random_state).fit(X_train, y_train)]
    
    def predict(self, X_test):
        # Prédictions spécifiques à chaque type de modèle
        if self.model_type == 'xgb':
            # Prédictions avec XGBoost
            if self.models is None:
                raise ValueError("Les modèles n'ont pas été entraînés. Veuillez appeler la méthode fit() avant de faire des prédictions.")
            return self.models[0].predict(X_test)
    
    def get_feature_importances(self):
        # Obtenir l'importance des caractéristiques spécifique à chaque type de modèle
        if self.model_type == 'xgb':
            # Obtenir l'importance des caractéristiques pour XGBoost
            # Exemple : self.models[0].feature_importances_
            pass




In [ ]:
class OptunaWeights:
    def __init__(self, random_state, n_trials=5000):
        self.study = None
        self.weights = None
        self.random_state = random_state
        self.n_trials = n_trials
    
    def _objective(self, trial, y_true, y_preds):
        # Objectif d'optimisation des poids
        weights = [trial.suggest_float(f"weight{n}", 0, 1) for n in range(len(y_preds))]
        weighted_pred = np.average(np.array(y_preds).T, axis=1, weights=weights)
        return roc_auc_score(y_true, weighted_pred)
    
    def fit(self, y_true, y_preds):
        optuna.logging.set_verbosity(optuna.logging.ERROR)
        sampler = CmaEsSampler(seed=self.random_state)
        pruner = HyperbandPruner()
        self.study = optuna.create_study(sampler=sampler, pruner=pruner, study_name="OptunaWeights", direction='maximize')
        objective_partial = partial(self._objective, y_true=y_true, y_preds=y_preds)
        self.study.optimize(objective_partial, n_trials=self.n_trials)
        self.weights = [self.study.best_params[f"weight{n}"] for n in range(len(y_preds))]
    
    def predict(self, y_preds):
        if self.weights is None:
            raise ValueError("Les poids n'ont pas été optimisés. Veuillez appeler la méthode fit() avant de faire des prédictions.")
        weighted_pred = np.average(np.array(y_preds).T, axis=1, weights=self.weights)
        return weighted_pred

In [ ]:
class ClassifierWithOptunaWeights:
    def __init__(self, model_type='xgb', n_estimators=100, random_state=0, n_trials=5000):
        self.classifier = Classifier(model_type=model_type, n_estimators=n_estimators, random_state=random_state)
        self.optuna_weights = OptunaWeights(random_state=random_state, n_trials=n_trials)
        
    def fit(self, X_train, y_train, y_preds):
        # Optimisation des hyperparamètres et validation croisée
        self.classifier.optimize_hyperparameters(X_train, y_train)
        self.classifier.cross_validate(X_train, y_train)
        
        # Entraînement du modèle de classification
        self.classifier.fit(X_train, y_train)
        
        # Optimisation des poids avec Optuna
        self.optuna_weights.fit(y_train, y_preds)
        
    def predict(self, X_test, y_preds):
        # Prédictions du modèle de classification
        classifier_predictions = self.classifier.predict(X_test)
        
        # Prédictions pondérées avec les poids optimisés
        weighted_predictions = self.optuna_weights.predict(y_preds)
        
        return classifier_predictions, weighted_predictions
    
    def fit_predict(self, X_train, y_train, X_test, y_preds):
        # Entraînement des modèles de classification et optimisation des poids
        self.fit(X_train, y_train, y_preds)
        
        # Prédictions sur l'ensemble de test
        classifier_predictions, weighted_predictions = self.predict(X_test, y_preds)
        
        return classifier_predictions, weighted_predictions

In [ ]:
class OptunaWeights:
    def __init__(self, random_state, n_trials=5000):
        self.study = None
        self.weights = None
        self.random_state = random_state
        self.n_trials = n_trials

    def _objective(self, trial, y_true, y_preds):
        # Define the weights for the predictions from each model
        weights = [trial.suggest_float(f"weight{n}", 0, 1) for n in range(len(y_preds))]

        # Calculate the weighted prediction
        weighted_pred = np.average(np.array(y_preds).T, axis=1, weights=weights)

        auc_score = roc_auc_score(y_true, weighted_pred)
        log_loss_score=log_loss(y_true, weighted_pred)
        return auc_score#/log_loss_score

    def fit(self, y_true, y_preds):
        optuna.logging.set_verbosity(optuna.logging.ERROR)
        sampler = optuna.samplers.CmaEsSampler(seed=self.random_state)
        pruner = optuna.pruners.HyperbandPruner()
        self.study = optuna.create_study(sampler=sampler, pruner=pruner, study_name="OptunaWeights", direction='maximize')
        objective_partial = partial(self._objective, y_true=y_true, y_preds=y_preds)
        self.study.optimize(objective_partial, n_trials=self.n_trials)
        self.weights = [self.study.best_params[f"weight{n}"] for n in range(len(y_preds))]

    def predict(self, y_preds):
        assert self.weights is not None, 'OptunaWeights error, must be fitted before predict'
        weighted_pred = np.average(np.array(y_preds).T, axis=1, weights=self.weights)
        return weighted_pred

    def fit_predict(self, y_true, y_preds):
        self.fit(y_true, y_preds)
        return self.predict(y_preds)
    
    def weights(self):
        return self.weights

In [ ]:

# Define a function to train a single model
def train_model(model, X_train_, y_train_, X_val, y_val, X_test):
    if 'ann' in str(model):
        model.fit(X_train_, y_train_, validation_data=(X_val, y_val), batch_size=4, epochs=5, verbose=verbose)
    else:
        model.fit(X_train_, y_train_, eval_set=[(X_val, y_val)], early_stopping_rounds=early_stopping_rounds, verbose=verbose)
    
    if 'ann' in str(model):
        test_pred = np.array(model.predict(X_test))[:, 0]
        y_val_pred = np.array(model.predict(X_val))[:, 0]
    else:
        test_pred = model.predict_proba(X_test)[:, 1]
        y_val_pred = model.predict_proba(X_val)[:, 1]

    score = roc_auc_score(y_val, y_val_pred)
    print(f'{str(model)} ROC AUC score: {score:.5f}')
    
    return test_pred, y_val_pred, score

# Define a function to perform cross-validation
def cross_validate(X_train, y_train, X_test, models):
    ensemble_score = []
    weights = []
    test_predss = np.zeros(X_test.shape[0])
    
    for i, (X_train_, X_val, y_train_, y_val) in enumerate(splitter.split_data(X_train, y_train, random_state_list=random_state_list)):
        oof_preds = []
        test_preds = []
        
        for name, model in models.items():
            test_pred, y_val_pred, score = train_model(model, X_train_, y_train_, X_val, y_val, X_test)
            oof_preds.append(y_val_pred)
            test_preds.append(test_pred)
        
        optweights = OptunaWeights(random_state=random_state)
        y_val_pred = optweights.fit_predict(y_val.values, oof_preds)
        score = roc_auc_score(y_val, y_val_pred)
        print(f'Ensemble ROC AUC score: {score:.5f}')
        ensemble_score.append(score)
        weights.append(optweights.weights)
        
        test_predss += optweights.predict(test_preds) / (n_splits * len(random_state_list))
        
        del X_train_, X_val, y_train_, y_val, oof_preds, test_preds
        gc.collect()

    return ensemble_score, weights, test_predss

# Perform cross-validation
# ensemble_score, weights, test_predss = cross_validate(X_train, y_train, X_test, models)

In [ ]:
def visualize_importance(models, feature_cols, title, head=15):
    # Initialize an empty list to store feature importance dataframes
    importance_dfs = []
    
    # Iterate over each model
    for i, model in enumerate(models):
        # Create a dataframe to store feature importance for this model
        _df = pd.DataFrame()
        _df["importance"] = model.feature_importances_
        _df["feature"] = pd.Series(feature_cols)
        _df["fold"] = i
        _df = _df.sort_values('importance', ascending=False).head(head)
        
        # Append the dataframe to the list
        importance_dfs.append(_df)
        
    # Concatenate all dataframes in the list into one dataframe
    feature_importance = pd.concat(importance_dfs, axis=0, ignore_index=True)
    
    # Group by feature and calculate the mean importance
    feature_importance_mean = feature_importance.groupby(["feature"])["importance"].mean().reset_index()
    
    # Sort by importance
    feature_importance_mean = feature_importance_mean.sort_values('importance', ascending=False)
    
    # Plot the feature importance
    plt.figure(figsize=(18, 10))
    sns.barplot(x='importance', y='feature', data=feature_importance_mean, color=(0.4, 0.76, 0.65), errorbar='sd')
    plt.xlabel('Importance', fontsize=14)
    plt.ylabel('Feature', fontsize=14)
    plt.title(f'{title} Feature Importance', fontsize=18)
    plt.grid(True, axis='x')
    plt.show()

## Annexe 

In [ ]:
### A revenir ici 
def high_freq_ohe(train, test, extra_cols, target, n_limit=50):
    '''
    If you wish to apply one hot encoding on a feature with so many unique values, then this can be applied, 
    where it takes a maximum of n categories and drops the rest of them treating as rare categories
    '''
    train_copy = train.copy()
    test_copy = test.copy()
    ohe_cols = []

    for col in extra_cols:
        # Calcul des valeurs rares
        rare_keys = train_copy[col].value_counts().nlargest(n_limit).index
        rare_key_map = dict(zip(rare_keys, np.full(len(rare_keys), 9999)))
        
        # Remplacement des valeurs rares
        train_copy[col] = train_copy[col].replace(rare_key_map)
        test_copy[col] = test_copy[col].replace(rare_key_map)
    
    # Encodage One-Hot
    train_copy, test_copy = OHE(train_copy, test_copy, extra_cols, target)
    
    # Suppression des colonnes avec des valeurs rares encodées et des colonnes uniques
    rare_cols = [col for col in train_copy.columns if "9999" in col or train_copy[col].nunique() == 1]
    train_copy = train_copy.drop(columns=rare_cols)
    test_copy = test_copy.drop(columns=rare_cols)
    
    return train_copy, test_copy

In [ ]:
### Reflechir et revenir sur cet encodage 
def high_freq_ohe(train, test, extra_cols, target, rare_threshold=0.05):
    '''
    Applique l'encodage one-hot en gérant les valeurs rares de manière dynamique.
    Utilise également des techniques d'encodage plus sophistiquées et la validation croisée pour le seuil de rareté.
    '''
    def encode_rare_values(df, col, rare_threshold):
        freq_counts = df[col].value_counts(normalize=True)
        rare_values = freq_counts[freq_counts < rare_threshold].index
        df[col] = df[col].apply(lambda x: "RARE_VALUE" if x in rare_values else x)
        return df

    def encode_categorical(df, cols):
        ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
        encoded_cols = pd.DataFrame(ohe.fit_transform(df[cols]), columns=ohe.get_feature_names_out(cols), index=df.index)
        df = df.drop(cols, axis=1)
        df = pd.concat([df, encoded_cols], axis=1)
        return df

    # Encodage des valeurs rares
    for col in extra_cols:
        train = encode_rare_values(train, col, rare_threshold)
        test = encode_rare_values(test, col, rare_threshold)

    # Encodage one-hot
    train = encode_categorical(train, extra_cols)
    test = encode_categorical(test, extra_cols)

    # Suppression des colonnes peu informatives
    train = train.drop(columns=train.columns[train.nunique() == 1])
    test = test.drop(columns=test.columns[test.nunique() == 1])

    return train, test

In [ ]:
### A revenir ulterieurement 
def encode_feature(train, test, feature, target):
    value_counts_dict = train[feature].value_counts().to_dict()  # Calculer une fois le dictionnaire des comptages
    train[feature + "_count"] = train[feature].map(value_counts_dict)
    test[feature + "_count"] = test[feature].map(value_counts_dict)
    
    train[feature+"_count_label"] = train[feature].replace({value: i 
                                                            for i, value in enumerate(value_counts_dict)}).astype(float)
    
    test[feature+"_count_label"] = test[feature].replace({value: i 
                                                          for i, value in enumerate(value_counts_dict)}).astype(float)
    
    if train[feature].dtype == 'O':
        if train[feature].nunique() <= 100:
            train, test = OHE(train, test, [feature], target)
        train = train.drop(columns=[feature])
        test = test.drop(columns=[feature])
    else:
        if train[feature].nunique() <= 20:
            train[feature] = train[feature].astype(str) + "_" + feature
            test[feature] = test[feature].astype(str) + "_" + feature
            train, test = OHE(train, test, [feature], target)
            train = train.drop(columns=[feature])
            test = test.drop(columns=[feature])
        else:
            train, test = high_freq_ohe(train, test, [feature], target, n_limit=20)
    return train, test

In [ ]:
def cluster_and_evaluate(train, cont_cols, target):
    # Splitting the data into train, validation, and test sets
    X_train, X_test, y_train, y_test = train_test_split(train[cont_cols], train[target], test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    # Scaling the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    # Grid search for hyperparameter optimization of DBSCAN
    param_grid = {'eps': [0.1, 0.5, 1.0],
                  'min_samples': [5, 10, 15]}
    grid_search = GridSearchCV(DBSCAN(), param_grid, cv=5)
    grid_search.fit(X_train_scaled)
    best_eps = grid_search.best_params_['eps']
    best_min_samples = grid_search.best_params_['min_samples']

    # Clustering with DBSCAN using the best hyperparameters
    dbscan = DBSCAN(eps=best_eps, min_samples=best_min_samples)
    cluster_labels_train = dbscan.fit_predict(X_train_scaled)
    cluster_labels_test = dbscan.fit_predict(X_test_scaled)

    # Silhouette score for evaluating clustering
    silhouette_train = silhouette_score(X_train_scaled, cluster_labels_train)
    silhouette_test = silhouette_score(X_test_scaled, cluster_labels_test)

    # Training a classifier using HistGradientBoostingClassifier
    model = HistGradientBoostingClassifier(max_iter=300, learning_rate=0.02, max_depth=6, random_state=42)
    model.fit(cluster_labels_train.reshape(-1, 1), y_train)

    # Evaluating the classifier on the validation set
    y_val_pred = model.predict_proba(dbscan.fit_predict(X_val_scaled).reshape(-1, 1))[:, 1]
    roc_auc_val = roc_auc_score(y_val, y_val_pred)

    # PrettyTable for displaying results
    table = PrettyTable()
    table.field_names = ['Metric', 'Value']
    table.add_row(['Silhouette Score (Train)', silhouette_train])
    table.add_row(['Silhouette Score (Test)', silhouette_test])
    table.add_row(['ROC AUC (Validation)', roc_auc_val])
    print(table)



In [ ]:
### A revenir là dessus 
def apply_arithmetic_operations(train_df, test_df, expressions_list):
    # Validate expressions
    valid_expressions = [expr for expr in expressions_list if expr.split()[0] in train_df.columns and expr.split()[2] in train_df.columns]
    
    for expression in valid_expressions:
        # Extract column names and operator
        col1, operator, col2 = expression.split()
        
        # Evaluate expression using numexpr for optimized performance
        train_df[expression] = evaluate(f'train_df["{col1}"] {operator} train_df["{col2}"]')
        test_df[expression] = evaluate(f'test_df["{col1}"] {operator} test_df["{col2}"]')

    return train_df, test_df

# Example usage:
train, test = apply_arithmetic_operations(train, test, ['col1 + col2', 'col1 - col2', 'col1 * col2', 'col1 / (col2 + 1e-5)'])

In [ ]:
def select_best_transformations(Xtrain, Xtest, cont_cols, Ytrain, threshold=0.90):
    final_drop_list = []  # Initialise une liste pour stocker les caractéristiques à supprimer
    table = PrettyTable()  # Initialise un objet PrettyTable pour afficher les résultats
    table.field_names = ['Original', 'Final Transformation', 'ROV AUC CV']  # Définit les noms de colonnes de la table
    best_cols = []  # Initialise une liste pour stocker les meilleures caractéristiques

    # Boucle sur chaque colonne continue dans cont_cols
    for col in cont_cols:
        # Sélectionne un sous-ensemble de colonnes contenant le nom de la colonne actuelle (col) et plus de 2 valeurs uniques
        sub_set = [f for f in Xtrain.columns if str(col) in str(f) and Xtrain[f].nunique() > 2]
        
        # Vérifie s'il y a suffisamment de colonnes dans le sous-ensemble
        if len(sub_set) > 2:
            correlated_features = []  # Initialise une liste pour stocker les caractéristiques fortement corrélées
            
            # Recherche des caractéristiques fortement corrélées
            for i, feature in enumerate(sub_set):
                for j in range(i + 1, len(sub_set)):
                    correlation = np.abs(Xtrain[feature].corr(Xtrain[sub_set[j]]))
                    # Ajoute les caractéristiques fortement corrélées à la liste
                    if correlation > threshold:
                        correlated_features.append(sub_set[j])

            correlated_features = list(set(correlated_features))  # Supprime les doublons de la liste
            
            # Vérifie s'il y a suffisamment de caractéristiques fortement corrélées
            if len(correlated_features) >= 2:
                temp_train = Xtrain[correlated_features]  # Sélectionne les colonnes de train corrélées
                temp_test = Xtest[correlated_features]  # Sélectionne les colonnes de test corrélées
                
                # Standardisation des données avant l'application de PCA
                sc = StandardScaler()
                temp_train = sc.fit_transform(temp_train)
                temp_test = sc.transform(temp_test)

                # Applique l'analyse en composantes principales (PCA) pour réduire la dimension
                pca = PCA(n_components=1)
                x_pca_train = pca.fit_transform(temp_train)
                x_pca_test = pca.transform(temp_test)
                x_pca_train = pd.DataFrame(x_pca_train, columns=[col+"_pca_comb_final"])
                x_pca_test = pd.DataFrame(x_pca_test, columns=[col+"_pca_comb_final"])
                train = pd.concat([Xtrain, x_pca_train], axis='columns')  # Ajoute les nouvelles colonnes PCA à train
                test = pd.concat([Xtest, x_pca_test], axis='columns')  # Ajoute les nouvelles colonnes PCA à test
                # Applique le clustering KMeans
                
                model = KMeans(n_clusters=10)
                kmeans = KMeans(n_clusters=10)
                kmeans.fit(np.array(temp_train))
                labels_train = kmeans.labels_
                Xtrain[col+'_final_cluster'] = labels_train
                Xtest[col+'_final_cluster'] = kmeans.predict(np.array(temp_test))
                correlated_features += [col+"_pca_comb_final", col+"_final_cluster"]  # Ajoute les nouvelles colonnes à la liste      
                # Évalue les performances de chaque caractéristique avec l'aire sous la courbe ROC (ROV AUC)
                
                kf = KFold(n_splits=5, shuffle=True, random_state=42)
                scores = []
                for f in correlated_features:
                    X = Xtrain[[f]].values
                    y = Ytrain.astype(int).values
                    auc = []
                    for train_idx, val_idx in kf.split(X, y):
                        X_train, y_train = X[train_idx], y[train_idx]
                        X_val, y_val = X[val_idx], y[val_idx]
                        # Modèle de classification utilisé pour évaluer la performance
                        model = HistGradientBoostingClassifier(max_iter=300, learning_rate=0.02, max_depth=6, random_state=42)
                        model.fit(X_train, y_train)
                        y_pred = model.predict_proba(X_val)[:, 1]
                        score = roc_auc_score(y_val, y_pred)
                        auc.append(score)
                    if f not in best_cols:
                        scores.append((f, np.mean(auc)))  # Ajoute le score moyen à la liste
                # Sélectionne la meilleure caractéristique basée sur le score ROV AUC
                best_col, best_auc = sorted(scores, key=lambda x: x[1], reverse=True)[0]
                best_cols.append(best_col)  # Ajoute la meilleure caractéristique à la liste
                # Identifie les caractéristiques à supprimer qui n'ont pas été sélectionnées
                cols_to_drop = [f for f in correlated_features if f not in best_cols]
                if cols_to_drop:
                    final_drop_list += cols_to_drop  # Ajoute les caractéristiques à la liste de suppression
                    
                # Ajoute une ligne à la table avec les résultats
                table.add_row([col, best_col, best_auc])

    print(table)  # Affiche la table des résultats
    return final_drop_list  # Retourne la liste des caractéristiques à supprimer

In [ ]:
def get_most_important_features(X_train, y_train, n, model_input):
    
    # Définir les paramètres pour la recherche aléatoire des hyperparamètres
    param_dist = {
        'n_estimators': randint(50, 300),
        'max_depth': randint(3, 10),
        'learning_rate': uniform(0.01, 0.3)
    }

    # Initialiser le modèle avec les paramètres de recherche aléatoire
    if 'xgb' in model_input:
        model = xgb.XGBClassifier(objective='binary:logistic', random_state=42, n_jobs=-1)
    elif 'cat' in model_input:
        model = CatBoostClassifier(random_state=42, verbose=False)
    else:
        model = lgb.LGBMClassifier(objective='binary', random_state=42, n_jobs=-1)

    # Effectuer la recherche aléatoire des hyperparamètres
    model_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=10, cv=5, scoring='roc_auc', random_state=42)
    model_search.fit(X_train, y_train)

    # Sélectionner les meilleurs hyperparamètres trouvés
    best_params = model_search.best_params_

    # Utiliser les meilleurs hyperparamètres pour initialiser le modèle final
    model.set_params(**best_params)

    # Entraîner le modèle final sur l'ensemble des données
    model.fit(X_train, y_train)

    # Extraire l'importance des fonctionnalités si disponible
    if hasattr(model, 'feature_importances_'):
        feature_importance_list = list(zip(X_train.columns, model.feature_importances_))
    elif hasattr(model, 'get_feature_importance'):
        feature_importance_list = list(zip(X_train.columns, model.get_feature_importance()))
    else:
        raise ValueError("Le modèle ne supporte pas l'importance des fonctionnalités.")

    # Trier les fonctionnalités par importance décroissante
    sorted_features = sorted(feature_importance_list, key=lambda x: x[1], reverse=True)
    
    # Sélectionner les n fonctionnalités les plus importantes
    top_n_features = [feature[0] for feature in sorted_features[:n]]
    
    return top_n_features

In [ ]:
def plot_feature_importance(top_n_features, avg_feature_importances, X_train, n):
    # Sélectionner les n fonctionnalités les plus importantes
    display_features = top_n_features[:10]

    # Tracer la barre horizontale pour les n fonctionnalités les plus importantes
    sns.set_palette("Set2")
    plt.figure(figsize=(8, 6))
    plt.barh(range(len(display_features)), [avg_feature_importances[X_train.columns.get_loc(feature)] for feature in display_features])
    plt.yticks(range(len(display_features)), display_features, fontsize=12)
    plt.xlabel('Importance moyenne de la fonctionnalité', fontsize=14)
    plt.ylabel('Fonctionnalités', fontsize=10)
    plt.title(f'Top {10} des {n} fonctionnalités les plus importantes', fontsize=16)
    plt.gca().invert_yaxis()  # Inverser l'axe y pour avoir la fonctionnalité la plus importante en haut
    plt.grid(axis='x', linestyle='--', alpha=0.7)
    plt.xticks(fontsize=8)
    plt.yticks(fontsize=8)

    # Ajouter les étiquettes de données sur les barres
    for index, value in enumerate([avg_feature_importances[X_train.columns.get_loc(feature)] for feature in display_features]):
        plt.text(value + 0.005, index, f'{value:.3f}', fontsize=12, va='center')

    plt.tight_layout()
    plt.show()

# Utilisation de la fonction plot_feature_importance pour tracer les fonctionnalités les plus importantes
#plot_feature_importance(top_n_features, avg_feature_importances, X_train, n)

n_imp_features_cat=get_most_important_features(Xtrain_final, Ytrain, 15, model_input ="cat")
n_imp_features_xgb=get_most_important_features(Xtrain_final, Ytrain, 15, model_input ="xgb")
n_imp_features_lgbm=get_most_important_features(Xtrain_final, Ytrain, 15, model_input ="lgb")